In [ ]:
#the basics
import pandas as pd, numpy as np
import math, json, gc, random, os, sys
from matplotlib import pyplot as plt
from tqdm import tqdm

#tensorflow deep learning basics
import tensorflow as tf
#import tensorflow_addons as tfa

import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Layer,Input,Activation, Lambda,Conv1D, SpatialDropout1D,Convolution1D,Dense,add,GlobalMaxPooling1D,GlobalAveragePooling1D,concatenate,Embedding
from tensorflow.keras.models import Model
from typing import List, Tuple
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split, KFold,  StratifiedKFold
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, ReLU, BatchNormalization, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU, Add


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Ensure plots are rendered inline (if using a Jupyter notebook)
%matplotlib inline

# -------------------------------
# 1. Load and Prepare the Dataset
# -------------------------------
file_path = "/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv"
df = pd.read_csv(file_path)

# Convert 'Timestamp' column to datetime for time-series analysis
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# List of numerical columns for later use
numerical_cols = ['Energy_Demand', 'Energy_Supply', 'Temperature', 'Grid_Load',
                  'Renewable_Source_Output', 'NonRenewable_Source_Output', 'Energy_Price']

# Set a common plot style
sns.set(style="whitegrid")

# -------------------------------
# 2. Time Series Analysis
# -------------------------------
plt.figure(figsize=(14, 6))
plt.plot(df['Timestamp'], df['Energy_Demand'], label='Energy Demand', color='blue')
plt.plot(df['Timestamp'], df['Energy_Supply'], label='Energy Supply', color='green')
plt.plot(df['Timestamp'], df['Temperature'], label='Temperature', color='red')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Time Series of Energy Demand, Energy Supply, and Temperature')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# -------------------------------
# 3. Distribution Analysis: Histograms & KDE Plots
# -------------------------------
# Histograms for numerical features
df[numerical_cols].hist(bins=15, figsize=(15, 10), layout=(3, 3))
plt.suptitle("Histograms of Numerical Features", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# KDE plots for numerical features (using fill=True instead of the deprecated shade=True)
plt.figure(figsize=(14, 8))
for col in numerical_cols:
    sns.kdeplot(data=df, x=col, fill=True, label=col)
plt.title('KDE Plot of Numerical Features')
plt.xlabel('Value')
plt.legend()
plt.show()

# -------------------------------
# 4. Correlation Analysis: Heatmap and Pairplot
# -------------------------------
# Compute correlation matrix for numerical columns
corr = df[numerical_cols].corr()

# Correlation Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

# Pairplot for pairwise relationships (this may take a moment)
sns.pairplot(df[numerical_cols])
plt.suptitle("Pairwise Relationships Between Numerical Features", y=1.02)
plt.show()

# -------------------------------
# 5. Categorical Analysis: Weather Conditions
# -------------------------------
# Count plot for Weather_Condition_x
plt.figure(figsize=(8, 4))
sns.countplot(x='Weather_Condition_x', data=df, palette='Set2')
plt.title('Count of Observations by Weather_Condition_x')
plt.xlabel('Weather Condition (x)')
plt.ylabel('Count')
plt.show()

# Count plot for Weather_Condition_y
plt.figure(figsize=(8, 4))
sns.countplot(x='Weather_Condition_y', data=df, palette='Set3')
plt.title('Count of Observations by Weather_Condition_y')
plt.xlabel('Weather Condition (y)')
plt.ylabel('Count')
plt.show()

# Box plots to see how weather conditions affect energy demand and supply (using Weather_Condition_x)
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='Weather_Condition_x', y='Energy_Demand', data=df, palette='pastel')
plt.title('Energy Demand by Weather_Condition_x')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
sns.boxplot(x='Weather_Condition_x', y='Energy_Supply', data=df, palette='pastel')
plt.title('Energy Supply by Weather_Condition_x')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# -------------------------------
# 6. Scatter Plots: Exploring Relationships
# -------------------------------
# Scatter plot: Grid Load vs. Energy Demand, colored by Weather_Condition_x
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Grid_Load', y='Energy_Demand', data=df, hue='Weather_Condition_x', palette='deep')
plt.title('Grid Load vs. Energy Demand')
plt.xlabel('Grid Load')
plt.ylabel('Energy Demand')
plt.legend(title='Weather Condition')
plt.show()

# Scatter plot: Renewable vs NonRenewable Source Output, colored by Energy Price.
# Since sns.scatterplot does not return a mappable for a colorbar, we create one manually.
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='Renewable_Source_Output',
    y='NonRenewable_Source_Output',
    data=df,
    hue='Energy_Price',
    palette='viridis',
    legend=False  # We'll add our own colorbar
)
plt.title('Renewable vs NonRenewable Source Output Colored by Energy Price')
plt.xlabel('Renewable Source Output')
plt.ylabel('NonRenewable Source Output')

# Create a normalization and ScalarMappable for the colorbar
norm = plt.Normalize(df['Energy_Price'].min(), df['Energy_Price'].max())
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])  # Required for older versions of matplotlib
cbar = plt.colorbar(sm, label='Energy Price')
plt.show()

# -------------------------------
# 7. Combined Time Series Plot with Dual Y-axis
# -------------------------------
fig, ax1 = plt.subplots(figsize=(14, 6))

# Plot energy metrics on the primary y-axis
ax1.plot(df['Timestamp'], df['Energy_Demand'], label='Energy Demand', color='blue')
ax1.plot(df['Timestamp'], df['Energy_Supply'], label='Energy Supply', color='green')
ax1.plot(df['Timestamp'], df['Temperature'], label='Temperature', color='red')
ax1.set_xlabel('Timestamp')
ax1.set_ylabel('Energy/Temperature Values')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='upper left')

# Plot Energy Price on a secondary y-axis
ax2 = ax1.twinx()
ax2.plot(df['Timestamp'], df['Energy_Price'], label='Energy Price', color='purple', linestyle='--')
ax2.set_ylabel('Energy Price')
ax2.legend(loc='upper right')

plt.title("Combined Time Series Plot: Energy Metrics and Price")
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd

# Specify the path to your dataset
file_path = "/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Print information about the DataFrame (such as column names, non-null counts, and data types)
print("Data Information:")
print(df.info())

# Print the first 5 rows of the DataFrame
print("\nFirst 5 Rows of Data:")
print(df.head())


In [ ]:
# Check unique values in both columns
print(df[['Weather_Condition_x', 'Weather_Condition_y']].drop_duplicates())

# Compare if they are the same
print((df['Weather_Condition_x'] == df['Weather_Condition_y']).value_counts())


In [ ]:
import pandas as pd

# Specify the path to your dataset
file_path = "/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Print basic DataFrame information
print("Data Information:")
df.info()  # This prints the info to stdout

# Print the first 5 rows of the DataFrame
print("\nFirst 5 Rows of Data:")
print(df.head())

# Convert the Timestamp column to datetime format for easier time-based operations
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
print("\nData Types After Converting Timestamp:")
print(df.dtypes)

# Print summary statistics for numeric columns
print("\nSummary Statistics:")
print(df.describe())

# If you want to inspect the weather condition columns, you can print unique values
print("\nUnique values in 'Weather_Condition_x':")
print(df['Weather_Condition_x'].unique())

print("\nUnique values in 'Weather_Condition_y':")
print(df['Weather_Condition_y'].unique())


In [ ]:
import matplotlib.pyplot as plt

# Plot Energy Demand over Time
plt.figure(figsize=(10, 6))
plt.plot(df['Timestamp'], df['Energy_Demand'], label='Energy Demand')
plt.xlabel('Time')
plt.ylabel('Energy Demand')
plt.title('Energy Demand Over Time')
plt.legend()
plt.show()


# 1. iTBG-Net: A Hybrid Deep Learning Model Combining Temporal Convolutional Networks and BiGRU

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from collections import deque
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU, Add, Dropout


#  Data Loading and Splitting into Training and Forecasting

data = pd.read_csv('/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.sort_values('Timestamp', inplace=True)


features = ['Energy_Demand']
sequence_length = 48  
prediction_length = 48  

n_total = len(data)
print ("The Total samples are in the dataset:", len(data))
n_forecast = prediction_length  # Reserving data for last 48 hours data for forecasting
n_train = n_total - n_forecast
training_data = data.iloc[:n_train].copy()
forecast_data = data.iloc[n_train:].copy()
print("The Samples for the Final forecasting are: " ,(n_forecast))
print ("The total Number of the train samples are:" ,(n_train))



#  Mean-Based Scaling

training_mean = training_data[features].mean().values[0]
training_features = training_data[features].values / training_mean
forecast_features = forecast_data[features].values / training_mean


#  Generate  Sequences from Training Data (Sliding Windows)

training_sequences, training_labels = [], []
for i in range(len(training_features) - sequence_length - prediction_length + 1):
    seq = training_features[i : i + sequence_length]
    label = training_features[i + sequence_length : i + sequence_length + prediction_length]
    training_sequences.append(seq)
    training_labels.append(label)

training_sequences = np.array(training_sequences)
training_labels = np.array(training_labels, dtype=np.float32)

# Define forecast sequence using the last sequence_length hours from training_features.
forecast_sequence = training_features[-sequence_length:]
forecast_label = forecast_features



#  Replay Buffer for Incremental Learning

class ReplayBuffer:
    def __init__(self, max_size=5000):
        self.buffer = deque(maxlen=max_size)
        
    def add(self, sequence, label):
        self.buffer.append((sequence, label))
        
    def sample(self, batch_size):
        if len(self.buffer) < batch_size:
            return list(self.buffer)
        else:
            return random.sample(self.buffer, batch_size)


#  Define TCN Block and Build the Hybrid Model 


def tcn_block(filters, kernel_size, dilation_rate, dropout):
    def block(x):
        # Residual connection to match dimensions.
        res = Conv1D(filters, kernel_size=1, padding="same")(x)
        conv1 = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding="causal", activation=None)(x)
        norm1 = BatchNormalization()(conv1)
        act1 = ReLU()(norm1)
        drop1 = Dropout(dropout)(act1)
        conv2 = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding="causal", activation=None)(drop1)
        norm2 = BatchNormalization()(conv2)
        act2 = ReLU()(norm2)
        drop2 = Dropout(dropout)(act2)
        skip = Add()([res, drop2])
        return skip
    return block

def build_hybrid_model(seq_len, pred_len, dropout=0.1, hidden_dim=64):
    inputs = tf.keras.layers.Input(shape=(seq_len, len(features)))
    tcn_output = tcn_block(hidden_dim, kernel_size=1, dilation_rate=1, dropout=dropout)(inputs)
    tcn_output = tcn_block(hidden_dim, kernel_size=1, dilation_rate=1, dropout=dropout)(tcn_output)
    
    gru_output = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal')
    )(tcn_output)
    
    lstm_output = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal')
    )(gru_output)
    
    
    truncated = lstm_output[:, :pred_len, :]
    outputs = tf.keras.layers.Dense(1, activation='linear')(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.0005),
        loss='mse',
        metrics=['mae', 'mse', tf.keras.metrics.MeanAbsolutePercentageError(name='mape')]
    )
    return model

# Build and visualize the new hybrid model 

hybrid_model = build_hybrid_model(seq_len=sequence_length, pred_len=prediction_length, dropout=0.1, hidden_dim=64)
hybrid_model.summary()
tf.keras.utils.plot_model(
    hybrid_model,
    to_file="hybrid_model_tcn_gru_lstm_architecture.png",
    show_shapes=True,
    show_layer_names=True
)


# Incremental Online Training Loop with Validation (Update-level Metrics)

class RegressionAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, tol=0.1):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.tol = tol
        self.train_accuracies = []
        self.val_accuracies = []
    
    def on_epoch_end(self, epoch, logs=None):
        train_preds = self.model.predict(self.train_data[0], verbose=0)
        train_acc = np.mean(np.abs(train_preds - self.train_data[1]) < self.tol)
        self.train_accuracies.append(train_acc)
        
        val_preds = self.model.predict(self.val_data[0], verbose=0)
        val_acc = np.mean(np.abs(val_preds - self.val_data[1]) < self.tol)
        self.val_accuracies.append(val_acc)

batch_size_update = 16      
epochs_per_update = 80      
replay_sample_size = 16

# We'll record one set of metrics per incremental update

metrics_history = {
    "incremental_update": [],
    "avg_loss": [],
    "avg_val_loss": [],
    "avg_accuracy": [],
    "avg_val_accuracy": [],
    "update_time": []
}

replay_buffer = ReplayBuffer(max_size=5000)

num_training_updates = int(np.ceil(len(training_sequences) / batch_size_update))
print(f"Starting incremental training over {num_training_updates} updates...\n")

for update_idx in range(0, len(training_sequences), batch_size_update):
    # Get new training samples for the current update
    new_sequences = training_sequences[update_idx:update_idx + batch_size_update]
    new_labels = training_labels[update_idx:update_idx + batch_size_update]
    update_length = len(new_sequences)
    
    # Determine time boundaries for logging (based on training_data timestamps)
    
    update_input_start = training_data['Timestamp'].iloc[update_idx]
    update_input_end = training_data['Timestamp'].iloc[min(update_idx + update_length + sequence_length - 1, len(training_data)-1)]
    update_label_start = training_data['Timestamp'].iloc[update_idx + sequence_length]
    update_label_end = training_data['Timestamp'].iloc[min(update_idx + update_length + sequence_length + prediction_length - 1, len(training_data)-1)]
    
    current_update = update_idx // batch_size_update + 1
    print(f"Processing Incremental update {current_update}/{num_training_updates}:")
    print(f"  - Training Input Hours: from {update_input_start} to {update_input_end}")
    print(f"  - Training Label Hours: from {update_label_start} to {update_label_end}")
    
    # Add new samples to the replay buffer
    
    for seq, lbl in zip(new_sequences, new_labels):
        replay_buffer.add(seq, lbl)
    
    # Sample from the replay buffer to mix with new data
    replay_samples = replay_buffer.sample(replay_sample_size)
    replay_sequences, replay_labels = zip(*replay_samples)
    replay_sequences = np.array(replay_sequences)
    replay_labels = np.array(replay_labels)
    
    # Combine new and replay data
    
    combined_sequences = np.vstack((new_sequences, replay_sequences))
    combined_labels = np.vstack((new_labels, replay_labels))
    
    # Split combined data into training and validation sets (80/20 split)
    
    total_combined = combined_sequences.shape[0]
    num_val = int(np.ceil(total_combined * 0.2))
    num_train = total_combined - num_val
    train_X = combined_sequences[:num_train]
    train_y = combined_labels[:num_train]
    val_X = combined_sequences[num_train:]
    val_y = combined_labels[num_train:]
    
    # Create a callback to compute regression accuracy per epoch
    
    acc_callback = RegressionAccuracyCallback(train_data=(train_X, train_y), val_data=(val_X, val_y), tol=0.1)
    
    start_time_update = time.time()
    history = hybrid_model.fit(
        train_X, train_y,
        batch_size=batch_size_update,
        epochs=epochs_per_update,
        verbose=0,
        validation_data=(val_X, val_y),
        shuffle=False,
        callbacks=[acc_callback]
    )
    end_time_update = time.time()
    
    update_time = end_time_update - start_time_update
    avg_loss = np.mean(history.history['loss'])
    avg_val_loss = np.mean(history.history['val_loss'])
    avg_accuracy = np.mean(acc_callback.train_accuracies)
    avg_val_accuracy = np.mean(acc_callback.val_accuracies)
    
    # Record update-level metrics
    metrics_history["incremental_update"].append(current_update)
    metrics_history["avg_loss"].append(avg_loss)
    metrics_history["avg_val_loss"].append(avg_val_loss)
    metrics_history["avg_accuracy"].append(avg_accuracy)
    metrics_history["avg_val_accuracy"].append(avg_val_accuracy)
    metrics_history["update_time"].append(update_time)
    
    print(f"Incremental update {current_update}/{num_training_updates}: "
          f"Avg Loss = {avg_loss:.4f}, "
          f"Avg Val Loss = {avg_val_loss:.4f}, "
          f"Update Time = {update_time:.2f}s\n")

# Save the trained model
model_save_path = "hybrid_model.h5"
hybrid_model.save(model_save_path)
print(f"Model saved to {model_save_path}\n")


# Forecasting and Plotting


predicted_values_ratio = hybrid_model.predict(np.expand_dims(forecast_sequence, axis=0)).flatten()
actual_values_ratio = forecast_label.flatten()

# Convert back to actual units
predicted_values = predicted_values_ratio * training_mean
actual_values = actual_values_ratio * training_mean

# Discard fractional parts by converting to integers.

predicted_values_int = predicted_values.astype(int)
actual_values_int = actual_values.astype(int)

# Print forecasted values along with the actual ground truth.

print("Forecasted Values:")
print(predicted_values_int)
print("Actual Ground Truth:")
print(actual_values_int)

fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
ax.plot(range(prediction_length), actual_values_int, label="Actual", marker="*", linewidth=1, color="firebrick")
ax.plot(range(prediction_length), predicted_values_int, label="Forecasted (iTBG-Net)", linestyle="--", marker=".", linewidth=1, color="blue")
ax.set_title("Actual vs Predicted Energy Demand (kWh)", fontsize=12, fontweight="bold")
ax.set_ylabel("Energy Consumption (kWh)", fontsize=10, fontweight="bold")
ax.legend(loc="upper right", fontsize=10)
ax.grid(True, linestyle="--", alpha=0.8)
ax.set_ylim(600, 1400)
plt.tight_layout()
plt.savefig("Final_Forecasting.eps", dpi=300, bbox_inches="tight")
plt.savefig("Final_Forecasting.pdf", dpi=300, bbox_inches="tight")
plt.savefig("Final_Forecasting.png", dpi=300, bbox_inches="tight")
plt.show()


#  Plot Training Loss per Incremental Update


plt.figure(figsize=(6, 4), dpi=300)
plt.plot(metrics_history["incremental_update"], metrics_history["avg_loss"], label="Train Loss", marker="*", linewidth=1, color="crimson")
plt.plot(metrics_history["incremental_update"], metrics_history["avg_val_loss"], label="Val Loss", marker=".",linewidth=1, color="royalblue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Loss", fontsize=10, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("PM_loss_per_update.png", dpi=300, bbox_inches="tight")
plt.show()


# Plot Regression Accuracy per Incremental Update


plt.figure(figsize=(6, 4), dpi=300)
plt.plot(metrics_history["incremental_update"], metrics_history["avg_accuracy"], label="Train Accuracy",  marker="*", linewidth=1, color="crimson")
plt.plot(metrics_history["incremental_update"], metrics_history["avg_val_accuracy"], label="Val Accuracy", marker=".", linewidth=1, color="royalblue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Regression Accuracy", fontsize=10, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("PM_regression_accuracy_per_update.png", dpi=300, bbox_inches="tight")
plt.show()


# Plot Training Time per Incremental Update 


updates_arr = np.array(metrics_history["incremental_update"])
update_times = np.array(metrics_history["update_time"])
selected_indices = np.arange(0, len(updates_arr), 2)  # adjust the step as needed
selected_updates = updates_arr[selected_indices]
selected_update_times = update_times[selected_indices]

plt.figure(figsize=(6, 4), dpi=300)
plt.plot(selected_updates, selected_update_times, marker="o", linestyle="-", markersize=5, linewidth=2,
         color="blue", markerfacecolor="white", markeredgewidth=1.2, markeredgecolor="blue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Training Time (seconds)", fontsize=10, fontweight="bold")
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("PM_Training_Time_per_update.png", dpi=300, bbox_inches="tight")
plt.show()


# Save the Performance Metrics for Later Comparison


np.savez("hybrid_metrics.npz",
         incremental_update=np.array(metrics_history["incremental_update"]),
         avg_loss=np.array(metrics_history["avg_loss"]),
         avg_val_loss=np.array(metrics_history["avg_val_loss"]),
         avg_accuracy=np.array(metrics_history["avg_accuracy"]),
         avg_val_accuracy=np.array(metrics_history["avg_val_accuracy"]),
         update_time=np.array(metrics_history["update_time"]),
         forecast_pred=np.array(predicted_values),
         actual=np.array(actual_values))
print("Hybrid model metrics saved to hybrid_metrics.npz")


# 2. CNN Model Incremental Learning

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from collections import deque
from tensorflow.keras.layers import Conv1D, BatchNormalization, Dropout
# Other necessary imports remain the same.

# ===========================
# 1. Data Loading and Splitting into Training and Forecasting
# ===========================
data = pd.read_csv('/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.sort_values('Timestamp', inplace=True)

# Use only the target column "Energy_Demand"
features = ['Energy_Demand']
sequence_length = 48  
prediction_length = 48  

n_total = len(data)
n_forecast = prediction_length  # Reserve the last 48 hours for forecasting
n_train = n_total - n_forecast

training_data = data.iloc[:n_train].copy()
forecast_data = data.iloc[n_train:].copy()

# ===========================
# 2. Mean-Based Scaling
# ===========================
training_mean = training_data[features].mean().values[0]
training_features = training_data[features].values / training_mean
forecast_features = forecast_data[features].values / training_mean

# ===========================
# 3. Generate Overlapping Sequences from Training Data
# ===========================
training_sequences, training_labels = [], []
for i in range(len(training_features) - sequence_length - prediction_length + 1):
    seq = training_features[i : i + sequence_length]
    label = training_features[i + sequence_length : i + sequence_length + prediction_length]
    training_sequences.append(seq)
    training_labels.append(label)

training_sequences = np.array(training_sequences)
training_labels = np.array(training_labels, dtype=np.float32)

# Define forecast sequence using the last sequence_length hours from training_features.
forecast_sequence = training_features[-sequence_length:]
forecast_label = forecast_features

# ===========================
# 4. Replay Buffer for Incremental Learning
# ===========================
class ReplayBuffer:
    def __init__(self, max_size=5000):
        self.buffer = deque(maxlen=max_size)
        
    def add(self, sequence, label):
        self.buffer.append((sequence, label))
        
    def sample(self, batch_size):
        if len(self.buffer) < batch_size:
            return list(self.buffer)
        else:
            return random.sample(self.buffer, batch_size)

# ===========================
# 5. Define the Complex CNN Model (Nearly 191K Parameters)
# ===========================
def build_cnn_model(seq_len=sequence_length, pred_len=prediction_length,
                    dropout=0.4, hidden_dim=128, dense_units=318):
    inputs = tf.keras.layers.Input(shape=(seq_len, len(features)))
    
    # --- Convolutional Block 1 ---
    x = Conv1D(filters=hidden_dim, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    
    # --- Convolutional Block 2 ---
    x = Conv1D(filters=hidden_dim, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    
    # --- Convolutional Block 3 ---
    x = Conv1D(filters=hidden_dim, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    
    # --- Convolutional Block 4 ---
    x = Conv1D(filters=hidden_dim, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    
    # --- TimeDistributed Dense Layers ---
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(dense_units, activation='relu'))(x)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))(x)
    
    # Truncate the output to ensure it matches the prediction length.
    outputs = tf.keras.layers.Lambda(lambda t: t[:, :pred_len, :])(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae', 'mse', tf.keras.metrics.MeanAbsolutePercentageError(name='mape')]
    )
    return model

cnn_model = build_cnn_model()
cnn_model.summary()
tf.keras.utils.plot_model(
    cnn_model,
    to_file="complex_cnn_model_architecture.png",
    show_shapes=True,
    show_layer_names=True
)

# ===========================
# 6. Incremental Online Training Loop with Validation (Update-level Metrics)
# ===========================
class RegressionAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, tol=0.1):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.tol = tol
        self.train_accuracies = []
        self.val_accuracies = []
    
    def on_epoch_end(self, epoch, logs=None):
        train_preds = self.model.predict(self.train_data[0], verbose=0)
        train_acc = np.mean(np.abs(train_preds - self.train_data[1]) < self.tol)
        self.train_accuracies.append(train_acc)
        
        val_preds = self.model.predict(self.val_data[0], verbose=0)
        val_acc = np.mean(np.abs(val_preds - self.val_data[1]) < self.tol)
        self.val_accuracies.append(val_acc)

batch_size_update = 16      
epochs_per_update = 80      
replay_sample_size = 16

# We'll record one set of metrics per incremental update
metrics_history = {
    "incremental_update": [],
    "avg_loss": [],
    "avg_val_loss": [],
    "avg_accuracy": [],
    "avg_val_accuracy": [],
    "update_time": []
}

replay_buffer = ReplayBuffer(max_size=5000)

num_training_updates = int(np.ceil(len(training_sequences) / batch_size_update))
print(f"Starting incremental training over {num_training_updates} updates...\n")

for update_idx in range(0, len(training_sequences), batch_size_update):
    # Get new training samples for the current update
    new_sequences = training_sequences[update_idx:update_idx + batch_size_update]
    new_labels = training_labels[update_idx:update_idx + batch_size_update]
    update_length = len(new_sequences)
    
    # Determine time boundaries for logging (based on training_data timestamps)
    update_input_start = training_data['Timestamp'].iloc[update_idx]
    update_input_end = training_data['Timestamp'].iloc[min(update_idx + update_length + sequence_length - 1, len(training_data)-1)]
    update_label_start = training_data['Timestamp'].iloc[update_idx + sequence_length]
    update_label_end = training_data['Timestamp'].iloc[min(update_idx + update_length + sequence_length + prediction_length - 1, len(training_data)-1)]
    
    current_update = update_idx // batch_size_update + 1
    print(f"Processing Incremental update {current_update}/{num_training_updates}:")
    print(f"  - Training Input Hours: from {update_input_start} to {update_input_end}")
    print(f"  - Training Label Hours: from {update_label_start} to {update_label_end}")
    
    # Add new samples to the replay buffer
    for seq, lbl in zip(new_sequences, new_labels):
        replay_buffer.add(seq, lbl)
    
    # Sample from the replay buffer to mix with new data
    replay_samples = replay_buffer.sample(replay_sample_size)
    replay_sequences, replay_labels = zip(*replay_samples)
    replay_sequences = np.array(replay_sequences)
    replay_labels = np.array(replay_labels)
    
    # Combine new and replay data
    combined_sequences = np.vstack((new_sequences, replay_sequences))
    combined_labels = np.vstack((new_labels, replay_labels))
    
    # Split combined data into training and validation sets (80/20 split)
    total_combined = combined_sequences.shape[0]
    num_val = int(np.ceil(total_combined * 0.2))
    num_train = total_combined - num_val
    train_X = combined_sequences[:num_train]
    train_y = combined_labels[:num_train]
    val_X = combined_sequences[num_train:]
    val_y = combined_labels[num_train:]
    
    # Create a callback to compute regression accuracy per epoch
    acc_callback = RegressionAccuracyCallback(train_data=(train_X, train_y), val_data=(val_X, val_y), tol=0.1)
    
    start_time_update = time.time()
    history = cnn_model.fit(
        train_X, train_y,
        batch_size=batch_size_update,
        epochs=epochs_per_update,
        verbose=0,
        validation_data=(val_X, val_y),
        shuffle=False,
        callbacks=[acc_callback]
    )
    end_time_update = time.time()
    
    update_time = end_time_update - start_time_update
    avg_loss = np.mean(history.history['loss'])
    avg_val_loss = np.mean(history.history['val_loss'])
    avg_accuracy = np.mean(acc_callback.train_accuracies)
    avg_val_accuracy = np.mean(acc_callback.val_accuracies)
    
    # Record update-level metrics
    metrics_history["incremental_update"].append(current_update)
    metrics_history["avg_loss"].append(avg_loss)
    metrics_history["avg_val_loss"].append(avg_val_loss)
    metrics_history["avg_accuracy"].append(avg_accuracy)
    metrics_history["avg_val_accuracy"].append(avg_val_accuracy)
    metrics_history["update_time"].append(update_time)
    
    print(f"Incremental update {current_update}/{num_training_updates}: "
          f"Avg Loss = {avg_loss:.4f}, "
          f"Avg Val Loss = {avg_val_loss:.4f}, "
          f"Update Time = {update_time:.2f}s\n")

# Save the trained model
model_save_path = "cnn_model.h5"
cnn_model.save(model_save_path)
print(f"Model saved to {model_save_path}\n")

# ===========================
# 7. Forecasting and Plotting
# ===========================
predicted_values_ratio = cnn_model.predict(np.expand_dims(forecast_sequence, axis=0)).flatten()
actual_values_ratio = forecast_label.flatten()

# Convert back to actual units
predicted_values = predicted_values_ratio * training_mean
actual_values = actual_values_ratio * training_mean

# Discard fractional parts by converting to integers.
predicted_values_int = predicted_values.astype(int)
actual_values_int = actual_values.astype(int)

# Print forecasted values along with the actual ground truth.
print("Forecasted Values:")
print(predicted_values_int)
print("Actual Ground Truth:")
print(actual_values_int)

fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
ax.plot(range(prediction_length), actual_values_int, label="Actual", marker="*", linewidth=1, color="firebrick")
ax.plot(range(prediction_length), predicted_values_int, label="Forecasted (CNN-IL)", linestyle="--", marker=".", linewidth=1, color="blue")
ax.set_title("Actual vs Predicted Energy Demand (kWh)", fontsize=12, fontweight="bold")
ax.set_ylabel("Energy Consumption (kWh)", fontsize=10, fontweight="bold")
ax.legend(loc="upper right", fontsize=10)
ax.grid(True, linestyle="--", alpha=0.8)
ax.set_ylim(600, 1400)
plt.tight_layout()
plt.savefig("Final_Forecasting_CNN.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 8. Plot Training Loss per Incremental Update
# ===========================
plt.figure(figsize=(6, 4), dpi=300)
plt.plot(metrics_history["incremental_update"], metrics_history["avg_loss"], label="Train Loss", marker="*", linewidth=1, color="crimson")
plt.plot(metrics_history["incremental_update"], metrics_history["avg_val_loss"], label="Val Loss", marker=".", linewidth=1, color="royalblue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Loss", fontsize=10, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("CNN_PM_loss_per_update.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 9. Plot Regression Accuracy per Incremental Update
# ===========================
plt.figure(figsize=(6, 4), dpi=300)
plt.plot(metrics_history["incremental_update"], metrics_history["avg_accuracy"], label="Train Accuracy",  marker="*", linewidth=1, color="crimson")
plt.plot(metrics_history["incremental_update"], metrics_history["avg_val_accuracy"], label="Val Accuracy", marker=".", linewidth=1, color="royalblue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Regression Accuracy", fontsize=10, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("CNN_PM_regression_accuracy_per_update.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 10. Plot Training Time per Incremental Update (Selected Updates)
# ===========================
updates_arr = np.array(metrics_history["incremental_update"])
update_times = np.array(metrics_history["update_time"])
selected_indices = np.arange(0, len(updates_arr), 2)  # adjust the step as needed
selected_updates = updates_arr[selected_indices]
selected_update_times = update_times[selected_indices]

plt.figure(figsize=(6, 4), dpi=300)
plt.plot(selected_updates, selected_update_times, marker="o", linestyle="-", markersize=5, linewidth=2,
         color="blue", markerfacecolor="white", markeredgewidth=1.2, markeredgecolor="blue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Training Time (seconds)", fontsize=10, fontweight="bold")
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("CNN_PM_Training_Time_per_update.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 11. Save the Performance Metrics for Later Comparison
# ===========================
np.savez("cnn_metrics.npz",
         incremental_update=np.array(metrics_history["incremental_update"]),
         avg_loss=np.array(metrics_history["avg_loss"]),
         avg_val_loss=np.array(metrics_history["avg_val_loss"]),
         avg_accuracy=np.array(metrics_history["avg_accuracy"]),
         avg_val_accuracy=np.array(metrics_history["avg_val_accuracy"]),
         update_time=np.array(metrics_history["update_time"]),
         forecast_pred=np.array(predicted_values),
         actual=np.array(actual_values))
print("CNN model metrics saved to cnn_metrics.npz")


# 3. LSTM Modeling Incremental Learning

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from collections import deque
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Lambda

# ===========================
# 1. Data Loading and Splitting into Training and Forecasting
# ===========================
data = pd.read_csv('/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.sort_values('Timestamp', inplace=True)

# Use only the target column "Energy_Demand"
features = ['Energy_Demand']
sequence_length = 48  
prediction_length = 48  

n_total = len(data)
n_forecast = prediction_length  # Reserve the last 48 hours for forecasting
n_train = n_total - n_forecast

training_data = data.iloc[:n_train].copy()
forecast_data = data.iloc[n_train:].copy()

# ===========================
# 2. Mean-Based Scaling
# ===========================
training_mean = training_data[features].mean().values[0]
training_features = training_data[features].values / training_mean
forecast_features = forecast_data[features].values / training_mean

# ===========================
# 3. Generate Overlapping Sequences from Training Data
# ===========================
training_sequences, training_labels = [], []
for i in range(len(training_features) - sequence_length - prediction_length + 1):
    seq = training_features[i : i + sequence_length]
    label = training_features[i + sequence_length : i + sequence_length + prediction_length]
    training_sequences.append(seq)
    training_labels.append(label)

training_sequences = np.array(training_sequences)
training_labels = np.array(training_labels, dtype=np.float32)

# Define forecast sequence using the last sequence_length hours from training_features.
forecast_sequence = training_features[-sequence_length:]
forecast_label = forecast_features

# ===========================
# 4. Replay Buffer for Incremental Learning
# ===========================
class ReplayBuffer:
    def __init__(self, max_size=5000):
        self.buffer = deque(maxlen=max_size)
        
    def add(self, sequence, label):
        self.buffer.append((sequence, label))
        
    def sample(self, batch_size):
        if len(self.buffer) < batch_size:
            return list(self.buffer)
        else:
            return random.sample(self.buffer, batch_size)

# ===========================
# 5. Define the Stacked LSTM Model (Nearly 191K Parameters)
# ===========================
def build_lstm_model(seq_len=sequence_length, pred_len=prediction_length, dropout=0.4):
    inputs = tf.keras.layers.Input(shape=(seq_len, len(features)))
    # First LSTM layer with 128 units.
    x = LSTM(128, return_sequences=True, dropout=dropout)(inputs)
    # Second LSTM layer with 123 units.
    x = LSTM(123, return_sequences=True, dropout=dropout)(x)
    # Map each timestep to the output value.
    x = TimeDistributed(Dense(1, activation='linear'))(x)
    # Truncate the output to ensure it matches the prediction length.
    outputs = Lambda(lambda t: t[:, :pred_len, :])(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae', 'mse', tf.keras.metrics.MeanAbsolutePercentageError(name='mape')]
    )
    return model

lstm_model = build_lstm_model()
lstm_model.summary()
tf.keras.utils.plot_model(
    lstm_model,
    to_file="lstm_model_architecture.png",
    show_shapes=True,
    show_layer_names=True
)

# ===========================
# 6. Incremental Online Training Loop with Validation (Update-level Metrics)
# ===========================
class RegressionAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, tol=0.1):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.tol = tol
        self.train_accuracies = []
        self.val_accuracies = []
    
    def on_epoch_end(self, epoch, logs=None):
        train_preds = self.model.predict(self.train_data[0], verbose=0)
        train_acc = np.mean(np.abs(train_preds - self.train_data[1]) < self.tol)
        self.train_accuracies.append(train_acc)
        
        val_preds = self.model.predict(self.val_data[0], verbose=0)
        val_acc = np.mean(np.abs(val_preds - self.val_data[1]) < self.tol)
        self.val_accuracies.append(val_acc)

batch_size_update = 16      
epochs_per_update = 80      
replay_sample_size = 16

# We'll record one set of metrics per incremental update.
metrics_history = {
    "incremental_update": [],
    "avg_loss": [],
    "avg_val_loss": [],
    "avg_accuracy": [],
    "avg_val_accuracy": [],
    "update_time": []
}

replay_buffer = ReplayBuffer(max_size=5000)

num_training_updates = int(np.ceil(len(training_sequences) / batch_size_update))
print(f"Starting incremental training over {num_training_updates} updates...\n")

for update_idx in range(0, len(training_sequences), batch_size_update):
    # Get new training samples for the current update.
    new_sequences = training_sequences[update_idx:update_idx + batch_size_update]
    new_labels = training_labels[update_idx:update_idx + batch_size_update]
    update_length = len(new_sequences)
    
    # Determine time boundaries for logging (based on training_data timestamps).
    update_input_start = training_data['Timestamp'].iloc[update_idx]
    update_input_end = training_data['Timestamp'].iloc[min(update_idx + update_length + sequence_length - 1, len(training_data)-1)]
    update_label_start = training_data['Timestamp'].iloc[update_idx + sequence_length]
    update_label_end = training_data['Timestamp'].iloc[min(update_idx + update_length + sequence_length + prediction_length - 1, len(training_data)-1)]
    
    current_update = update_idx // batch_size_update + 1
    print(f"Processing Incremental update {current_update}/{num_training_updates}:")
    print(f"  - Training Input Hours: from {update_input_start} to {update_input_end}")
    print(f"  - Training Label Hours: from {update_label_start} to {update_label_end}")
    
    # Add new samples to the replay buffer.
    for seq, lbl in zip(new_sequences, new_labels):
        replay_buffer.add(seq, lbl)
    
    # Sample from the replay buffer to mix with new data.
    replay_samples = replay_buffer.sample(replay_sample_size)
    replay_sequences, replay_labels = zip(*replay_samples)
    replay_sequences = np.array(replay_sequences)
    replay_labels = np.array(replay_labels)
    
    # Combine new and replay data.
    combined_sequences = np.vstack((new_sequences, replay_sequences))
    combined_labels = np.vstack((new_labels, replay_labels))
    
    # Split combined data into training and validation sets (80/20 split).
    total_combined = combined_sequences.shape[0]
    num_val = int(np.ceil(total_combined * 0.2))
    num_train = total_combined - num_val
    train_X = combined_sequences[:num_train]
    train_y = combined_labels[:num_train]
    val_X = combined_sequences[num_train:]
    val_y = combined_labels[num_train:]
    
    # Create a callback to compute regression accuracy per epoch.
    acc_callback = RegressionAccuracyCallback(train_data=(train_X, train_y), val_data=(val_X, val_y), tol=0.1)
    
    start_time_update = time.time()
    history = lstm_model.fit(
        train_X, train_y,
        batch_size=batch_size_update,
        epochs=epochs_per_update,
        verbose=0,
        validation_data=(val_X, val_y),
        shuffle=False,
        callbacks=[acc_callback]
    )
    end_time_update = time.time()
    
    update_time = end_time_update - start_time_update
    avg_loss = np.mean(history.history['loss'])
    avg_val_loss = np.mean(history.history['val_loss'])
    avg_accuracy = np.mean(acc_callback.train_accuracies)
    avg_val_accuracy = np.mean(acc_callback.val_accuracies)
    
    # Record update-level metrics.
    metrics_history["incremental_update"].append(current_update)
    metrics_history["avg_loss"].append(avg_loss)
    metrics_history["avg_val_loss"].append(avg_val_loss)
    metrics_history["avg_accuracy"].append(avg_accuracy)
    metrics_history["avg_val_accuracy"].append(avg_val_accuracy)
    metrics_history["update_time"].append(update_time)
    
    print(f"Incremental update {current_update}/{num_training_updates}: "
          f"Avg Loss = {avg_loss:.4f}, "
          f"Avg Val Loss = {avg_val_loss:.4f}, "
          f"Update Time = {update_time:.2f}s\n")

# Save the trained model.
model_save_path = "lstm_model.h5"
lstm_model.save(model_save_path)
print(f"Model saved to {model_save_path}\n")

# ===========================
# 7. Forecasting and Plotting
# ===========================
predicted_values_ratio = lstm_model.predict(np.expand_dims(forecast_sequence, axis=0)).flatten()
actual_values_ratio = forecast_label.flatten()

# Convert back to actual units.
predicted_values = predicted_values_ratio * training_mean
actual_values = actual_values_ratio * training_mean

# Discard fractional parts by converting to integers.
predicted_values_int = predicted_values.astype(int)
actual_values_int = actual_values.astype(int)

# Print forecasted values along with the actual ground truth.
print("Forecasted Values:")
print(predicted_values_int)
print("Actual Ground Truth:")
print(actual_values_int)

fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
ax.plot(range(prediction_length), actual_values_int, label="Actual", marker="*", linewidth=1, color="firebrick")
ax.plot(range(prediction_length), predicted_values_int, label="Forecasted (LSTM-IL)", linestyle="--", marker=".", linewidth=1, color="blue")
ax.set_title("Actual vs Predicted Energy Demand (kWh)", fontsize=12, fontweight="bold")
ax.set_ylabel("Energy Consumption (kWh)", fontsize=10, fontweight="bold")
ax.legend(loc="upper right", fontsize=10)
ax.grid(True, linestyle="--", alpha=0.8)
ax.set_ylim(600, 1400)
plt.tight_layout()
plt.savefig("Final_Forecasting_LSTM.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 8. Plot Training Loss per Incremental Update
# ===========================
plt.figure(figsize=(6, 4), dpi=300)
plt.plot(metrics_history["incremental_update"], metrics_history["avg_loss"], label="Train Loss", marker="*", linewidth=1, color="crimson")
plt.plot(metrics_history["incremental_update"], metrics_history["avg_val_loss"], label="Val Loss", marker=".", linewidth=1, color="royalblue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Loss", fontsize=10, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("LSTM_PM_loss_per_update.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 9. Plot Regression Accuracy per Incremental Update
# ===========================
plt.figure(figsize=(6, 4), dpi=300)
plt.plot(metrics_history["incremental_update"], metrics_history["avg_accuracy"], label="Train Accuracy",  marker="*", linewidth=1, color="crimson")
plt.plot(metrics_history["incremental_update"], metrics_history["avg_val_accuracy"], label="Val Accuracy", marker=".", linewidth=1, color="royalblue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Regression Accuracy", fontsize=10, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("LSTM_PM_regression_accuracy_per_update.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 10. Plot Training Time per Incremental Update (Selected Updates)
# ===========================
updates_arr = np.array(metrics_history["incremental_update"])
update_times = np.array(metrics_history["update_time"])
selected_indices = np.arange(0, len(updates_arr), 2)  # Adjust the step as needed.
selected_updates = updates_arr[selected_indices]
selected_update_times = update_times[selected_indices]

plt.figure(figsize=(6, 4), dpi=300)
plt.plot(selected_updates, selected_update_times, marker="o", linestyle="-", markersize=5, linewidth=2,
         color="blue", markerfacecolor="white", markeredgewidth=1.2, markeredgecolor="blue")
plt.xlabel("Incremental update", fontsize=10, fontweight="bold")
plt.ylabel("Training Time (seconds)", fontsize=10, fontweight="bold")
plt.grid(True, linestyle="--", alpha=0.8)
plt.tight_layout()
plt.savefig("LSTM_PM_Training_Time_per_update.png", dpi=300, bbox_inches="tight")
plt.show()

# ===========================
# 11. Save the Performance Metrics for Later Comparison
# ===========================
np.savez("lstm_metrics.npz",
         incremental_update=np.array(metrics_history["incremental_update"]),
         avg_loss=np.array(metrics_history["avg_loss"]),
         avg_val_loss=np.array(metrics_history["avg_val_loss"]),
         avg_accuracy=np.array(metrics_history["avg_accuracy"]),
         avg_val_accuracy=np.array(metrics_history["avg_val_accuracy"]),
         update_time=np.array(metrics_history["update_time"]),
         forecast_pred=np.array(predicted_values),
         actual=np.array(actual_values))
print("LSTM model metrics saved to lstm_metrics.npz")


**xLSTM Modeling Importing**

In [ ]:
# Clone the XLSTM repository
!git clone https://github.com/NX-AI/xlstm.git

# Navigate to the xlstm directory
%cd xlstm




In [ ]:
# Install PyTorch
!pip install torch torchvision torchaudio

# Install additional dependencies
!pip install numpy pandas scikit-learn matplotlib

# Install the xLSTM package
!pip install xlstm


In [ ]:
#!pip install seaborn


In [ ]:
from xlstm.xlstm.blocks.mlstm.layer import mLSTMLayerConfig
from xlstm.xlstm.blocks.slstm.layer import sLSTMLayerConfig
from xlstm.xlstm.xlstm_block_stack import xLSTMBlockStack, xLSTMBlockStackConfig
from xlstm.xlstm.blocks.mlstm.block import mLSTMBlockConfig
from xlstm.xlstm.blocks.slstm.block import sLSTMBlockConfig
from xlstm.xlstm.components.feedforward import FeedForwardConfig

In [ ]:
!apt-get install ninja-build -y


In [ ]:
!ninja --version

In [ ]:
!nvcc --version


In [ ]:
#!rm -rf /root/.cache/torch_extensions/


In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torchinfo


In [ ]:
try:
    from torchinfo import summary  # For model summary
except ImportError:
    !pip install torchinfo
    from torchinfo import summary


# 4. xLSTM Incremental Learning Model

In [ ]:
import os
import time
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Subset
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from torchinfo import summary

# Import xLSTM components
from xlstm.xlstm.blocks.slstm.layer import sLSTMLayerConfig
from xlstm.xlstm.blocks.slstm.block import sLSTMBlockConfig
from xlstm.xlstm.blocks.mlstm.layer import mLSTMLayerConfig
from xlstm.xlstm.blocks.mlstm.block import mLSTMBlockConfig
from xlstm.xlstm.xlstm_block_stack import xLSTMBlockStack, xLSTMBlockStackConfig

# =============================================================================
# 1. Data Loading, Early Splitting, and Preprocessing (Mean-Based Scaling)
# =============================================================================
def load_energy_dataset(file_path, sequence_length=48, prediction_length=48):
    # Read CSV and preprocess timestamps.
    data = pd.read_csv(file_path)
    data['Datetime'] = pd.to_datetime(data['Timestamp'])
    # Drop unused columns and sort by time.
    data = data.drop(columns=['Timestamp', 'Weather_Condition_x', 'Weather_Condition_y'])
    data = data.sort_values(by='Datetime').reset_index(drop=True)
    numeric_columns = ['Energy_Demand', 'Energy_Supply', 'Temperature', 'Grid_Load',
                       'Renewable_Source_Output', 'NonRenewable_Source_Output', 'Energy_Price']
    data = data[['Datetime'] + numeric_columns]
    # Resample hourly and forward fill missing values.
    data_hourly = data.set_index('Datetime').resample('h').mean().ffill()
    
    # Early Split: Reserve the last prediction_length hours for forecasting.
    total_rows = len(data_hourly)
    n_forecast = prediction_length  # e.g., last 48 rows reserved for forecasting
    n_train = total_rows - n_forecast
    train_data = data_hourly.iloc[:n_train]
    forecast_data = data_hourly.iloc[n_train:]
    
    print(f"Total records in resampled dataset: {total_rows}")
    print(f"Training records: {n_train}")
    print(f"Forecast records: {n_forecast}")
    
    # Scale 'Energy_Demand' using mean-based scaling (divide by training mean).
    energy_demand_train = train_data['Energy_Demand'].values.reshape(-1, 1)
    training_mean = energy_demand_train.mean()  # Compute training mean
    train_scaled = energy_demand_train / training_mean

    # Scale forecast data using the same training mean.
    energy_demand_forecast = forecast_data['Energy_Demand'].values.reshape(-1, 1)
    forecast_scaled = energy_demand_forecast / training_mean
    
    # Generate sliding-window sequences from training data.
    X, y = [], []
    num_sequences = len(train_scaled) - sequence_length - prediction_length + 1
    for i in range(num_sequences):
        X.append(train_scaled[i : i + sequence_length])
        y.append(train_scaled[i + sequence_length : i + sequence_length + prediction_length])
    X = torch.tensor(X, dtype=torch.float32)  # shape: (num_sequences, sequence_length, 1)
    y = torch.tensor(y, dtype=torch.float32).squeeze(-1)  # shape: (num_sequences, prediction_length)
    
    print(f"Total training sequences generated: {len(X)}")
    
    # Create forecast sequence from the last sequence_length hours of training data.
    forecast_sequence = train_scaled[-sequence_length:]
    forecast_sequence = torch.tensor(forecast_sequence, dtype=torch.float32)
    
    # Forecast labels from reserved forecast data.
    forecast_labels = torch.tensor(forecast_scaled, dtype=torch.float32).squeeze(-1)
    
    train_loader = DataLoader(TensorDataset(X, y), batch_size=32, shuffle=True)
    return train_loader, training_mean, train_data, forecast_sequence, forecast_labels

# =============================================================================
# 2. Split Data into Chunks for Incremental Training
# =============================================================================
def split_data_into_chunks(data_loader, chunk_size):
    data_chunks = []
    X, y = data_loader.dataset.tensors
    for i in range(0, len(X), chunk_size):
        chunk_X = X[i : i + chunk_size]
        chunk_y = y[i : i + chunk_size]
        if len(chunk_X) > 0:
            data_chunks.append((i, DataLoader(TensorDataset(chunk_X, chunk_y), batch_size=32, shuffle=True)))
    return data_chunks

# =============================================================================
# 3. Define the XLSTM Model (Reduced Capacity)
# =============================================================================
class XLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, slstm_blocks=1, mlstm_blocks=1):
        super(XLSTMModel, self).__init__()
        self.input_embedding = nn.Linear(input_size, hidden_size)
        self.blocks = nn.ModuleList()
        block_idx = 0
        # Add one sLSTM block.
        while slstm_blocks > 0:
            slstm_layer_config = sLSTMLayerConfig(
                hidden_size=hidden_size,
                num_heads=4,
                num_states=3,
                backend='cuda',
                function='slstm',
                dropout=0.4
            )
            slstm_block_config = sLSTMBlockConfig(
                slstm=slstm_layer_config,
                _num_blocks=1,
                _block_idx=block_idx
            )
            self.blocks.append(
                xLSTMBlockStack(config=xLSTMBlockStackConfig(
                    slstm_block=slstm_block_config,
                    num_blocks=1,
                    context_length=48,
                    embedding_dim=hidden_size,
                    dropout=0.4,
                    add_post_blocks_norm=False
                ))
            )
            slstm_blocks -= 1
            block_idx += 1
        # Add one mLSTM block.
        while mlstm_blocks > 0:
            mlstm_layer_config = mLSTMLayerConfig(
                num_heads=1,
                embedding_dim=hidden_size,
                dropout=0.4
            )
            mlstm_block_config = mLSTMBlockConfig(
                mlstm=mlstm_layer_config,
                _num_blocks=1,
                _block_idx=block_idx
            )
            self.blocks.append(
                xLSTMBlockStack(config=xLSTMBlockStackConfig(
                    mlstm_block=mlstm_block_config,
                    num_blocks=1,
                    context_length=48,
                    embedding_dim=hidden_size,
                    dropout=0.4,
                    add_post_blocks_norm=False
                ))
            )
            mlstm_blocks -= 1
            block_idx += 1
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: (batch_size, context_length, input_size)
        x = self.input_embedding(x)  # (batch_size, 48, hidden_size)
        for block in self.blocks:
            x = block(x)
        # Use the final time step for prediction.
        x = self.fc(x[:, -1, :])
        return x

# =============================================================================
# 4. Regression Accuracy Computation (Tolerance-Based)
# =============================================================================
def compute_regression_accuracy(model, loader, tol, device):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_X, batch_y in loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            preds = model(batch_X)
            all_preds.append(preds.cpu().numpy())
            all_targets.append(batch_y.cpu().numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    return np.mean(np.abs(all_preds - all_targets) < tol)

# =============================================================================
# 5. Incremental Training Functions
# =============================================================================
def train_one_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    return total_loss / len(loader) if len(loader) > 0 else 0

def split_train_val(chunk_loader, val_ratio=0.1):
    dataset = chunk_loader.dataset
    total_samples = len(dataset)
    indices = list(range(total_samples))
    split = int(val_ratio * total_samples)
    if total_samples > 1 and split < 1:
        split = 1
    random.shuffle(indices)
    val_indices = indices[:split]
    train_indices = indices[split:]
    if len(train_indices) == 0:
        train_indices = indices
        val_indices = indices
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    return train_loader, val_loader

# =============================================================================
# 6. Main Incremental Training Script for xLSTM Model (Tracking Performance per Update)
# =============================================================================
if __name__ == '__main__':
    file_path = '/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv'
    # load_energy_dataset returns: train_loader, training_mean, train_data, forecast_sequence, forecast_labels.
    train_loader, training_mean, train_data, forecast_sequence, forecast_labels = load_energy_dataset(
        file_path, sequence_length=48, prediction_length=48
    )
    chunk_size = 16  # Number of new samples per incremental update.
    data_chunks = split_data_into_chunks(train_loader, chunk_size=chunk_size)
    total_chunks = len(data_chunks)
    print(f"Total number of incremental updates (chunks): {total_chunks}")

    # Model configuration (reduced capacity to match previous models)
    input_size = 1
    hidden_size = 128       # To match comparable capacity.
    output_size = 48        # Prediction horizon.
    slstm_blocks = 1        # 1 sLSTM block.
    mlstm_blocks = 1        # 1 mLSTM block.

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    modelxlstm = XLSTMModel(input_size, hidden_size, output_size,
                            slstm_blocks=slstm_blocks, mlstm_blocks=mlstm_blocks).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(modelxlstm.parameters(), lr=0.001)

    # Print model summary using dummy input.
    print(summary(modelxlstm, input_size=(32, 48, 1)))

    # Training configuration: local epochs per update and tolerance.
    chunk_epochs = 80
    tol = 0.1

    # Metrics history dictionary for incremental updates.
    metrics_history = {
        "incremental_update": [],
        "avg_loss": [],
        "avg_val_loss": [],
        "avg_accuracy": [],
        "avg_val_accuracy": [],
        "update_time": []
    }

    print("Starting incremental training on xLSTM model (tracking performance per update)...")
    update_count = 0

    for chunk_index, (chunk_start_idx, chunk_loader) in enumerate(data_chunks):
        chunk_len = len(chunk_loader.dataset)
        # Determine time window from train_data index.
        start_time = train_data.index[chunk_start_idx]
        end_idx = chunk_start_idx + chunk_len + 48 - 1
        end_time = train_data.index[end_idx] if end_idx < len(train_data.index) else train_data.index[-1]
        print(f"\nUpdate {chunk_index+1}/{total_chunks} - New Data: {chunk_len} samples covering hours from {start_time} to {end_time}")
        
        train_chunk_loader, val_loader = split_train_val(chunk_loader, val_ratio=0.1)
        epoch_train_losses = []
        epoch_val_losses = []
        epoch_train_accuracies = []
        epoch_val_accuracies = []
        update_start_time = time.time()
        
        for epoch in range(chunk_epochs):
            start_epoch = time.time()
            train_loss = train_one_epoch(modelxlstm, train_chunk_loader, optimizer, criterion, device)
            val_loss = evaluate(modelxlstm, val_loader, criterion, device)
            train_acc = compute_regression_accuracy(modelxlstm, train_chunk_loader, tol, device)
            val_acc = compute_regression_accuracy(modelxlstm, val_loader, tol, device)
            epoch_train_losses.append(train_loss)
            epoch_val_losses.append(val_loss)
            epoch_train_accuracies.append(train_acc)
            epoch_val_accuracies.append(val_acc)
            end_epoch = time.time()
            # Uncomment the following line if you need per-epoch reporting.
            # print(f"  [Update {chunk_index+1}] Epoch {epoch+1}/{chunk_epochs}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}, Epoch Time={(end_epoch-start_epoch):.2f}s")
        
        update_end_time = time.time()
        update_time = update_end_time - update_start_time
        update_count += 1
        avg_train_loss = np.mean(epoch_train_losses)
        avg_val_loss = np.mean(epoch_val_losses)
        avg_train_acc = np.mean(epoch_train_accuracies)
        avg_val_acc = np.mean(epoch_val_accuracies)
        
        metrics_history["incremental_update"].append(update_count)
        metrics_history["avg_loss"].append(avg_train_loss)
        metrics_history["avg_val_loss"].append(avg_val_loss)
        metrics_history["avg_accuracy"].append(avg_train_acc)
        metrics_history["avg_val_accuracy"].append(avg_val_acc)
        metrics_history["update_time"].append(update_time)
        
        print(f"Update {update_count} complete: Avg Train Loss={avg_train_loss:.4f}, Avg Val Loss={avg_val_loss:.4f}, "
              f"Avg Train Acc={avg_train_acc:.4f}, Avg Val Acc={avg_val_acc:.4f}, Update Time={update_time:.2f}s")
        
        modelxlstm.eval()
        all_preds, all_targets = [], []
        with torch.no_grad():
            for batch_X, batch_y in chunk_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                preds = modelxlstm(batch_X).cpu().numpy()
                all_preds.extend(preds)
                all_targets.extend(batch_y.cpu().numpy())
        mae = mean_absolute_error(all_targets, all_preds)
        mse = mean_squared_error(all_targets, all_preds)
        r2 = r2_score(all_targets, all_preds)
        print(f"Update {update_count} Evaluation - MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")

    print("Incremental training complete.\n")

    # =============================================================================
    # 7. Forecasting Visualization: Compare Actual vs. Predicted Sequences (Original Units)
    # =============================================================================
    modelxlstm.eval()
    with torch.no_grad():
        predicted_values = modelxlstm(forecast_sequence.unsqueeze(0).to(device)).cpu().numpy().flatten()
    actual_values = forecast_labels.flatten().numpy()
    # Convert normalized forecasts back to original energy consumption (kWh).
    predicted_values = predicted_values * training_mean
    actual_values = actual_values * training_mean

    fig, ax = plt.subplots(figsize=(8, 5), dpi=300)
    ax.plot(range(48), actual_values, label="Actual", color="firebrick", marker="*", linewidth=1)
    ax.plot(range(48), predicted_values, label="Forecast (XLSTM)", color="blue", linestyle="--", marker=".", linewidth=1)
    ax.set_title("Actual vs. Predicted Energy Demand (kWh)", fontsize=12, fontweight="bold")
    ax.set_xlabel("Time Steps (Hours)", fontsize=10, fontweight="bold")
    ax.set_ylabel("Energy Consumption (kWh)", fontsize=10, fontweight="bold")
    ax.legend(loc="upper right", fontsize=10)
    ax.grid(True, linestyle="--", alpha=0.8)
    plt.tight_layout()
    plt.savefig("Final_Forecasting_XLSTM.png", dpi=300, bbox_inches="tight")
    plt.show()

    # =============================================================================
    # 8. Plot Training and Validation Loss per Update
    # =============================================================================
    updates_arr = np.array(metrics_history["incremental_update"])
    plt.figure(figsize=(6, 4), dpi=300)
    plt.plot(updates_arr, metrics_history["avg_loss"], label="Train Loss", color="crimson", linewidth=1)
    plt.plot(updates_arr, metrics_history["avg_val_loss"], label="Validation Loss", color="royalblue", linewidth=1)
    plt.title("XLSTM Model: Training and Validation Loss per Update", fontsize=10, fontweight="bold")
    plt.xlabel("Incremental Update", fontsize=10, fontweight="bold")
    plt.ylabel("Loss", fontsize=10, fontweight="bold")
    plt.legend(fontsize=10)
    plt.grid(True, linestyle="--", alpha=0.8)
    plt.tight_layout()
    plt.savefig("XLSTM_loss_per_update.png", dpi=300, bbox_inches="tight")
    plt.show()

    # =============================================================================
    # 9. Plot Regression Accuracy per Update (Tolerance-Based)
    # =============================================================================
    plt.figure(figsize=(6, 4), dpi=300)
    plt.plot(updates_arr, metrics_history["avg_accuracy"], label="Train Accuracy", color="crimson", linewidth=1)
    plt.plot(updates_arr, metrics_history["avg_val_accuracy"], label="Validation Accuracy", color="royalblue", linewidth=1)
    plt.title("XLSTM Model: Training and Validation Regression Accuracy per Update", fontsize=10, fontweight="bold")
    plt.xlabel("Incremental Update", fontsize=10, fontweight="bold")
    plt.ylabel("Regression Accuracy", fontsize=10, fontweight="bold")
    plt.legend(fontsize=10)
    plt.grid(True, linestyle="--", alpha=0.8)
    plt.tight_layout()
    plt.savefig("XLSTM_regression_accuracy_per_update.png", dpi=300, bbox_inches="tight")
    plt.show()

    # =============================================================================
    # 10. Plot Training Time per Incremental Update (Selected Updates)
    # =============================================================================
    update_times_arr = np.array(metrics_history["update_time"])
    selected_indices = np.arange(0, len(updates_arr), 2)  # Adjust step size as needed.
    selected_updates = updates_arr[selected_indices]
    selected_update_times = update_times_arr[selected_indices]

    plt.figure(figsize=(6, 4), dpi=300)
    plt.plot(selected_updates, selected_update_times, marker="o", linestyle="-", markersize=5, linewidth=2,
             color="blue", markerfacecolor="white", markeredgewidth=1.2, markeredgecolor="blue")
    plt.xlabel("Incremental Update", fontsize=10, fontweight="bold")
    plt.ylabel("Training Time (seconds)", fontsize=10, fontweight="bold")
    plt.title("XLSTM Model: Training Time per Incremental Update", fontsize=10, fontweight="bold", pad=5)
    plt.grid(True, linestyle="--", alpha=0.8)
    plt.tight_layout()
    plt.savefig("XLSTM_Training_Time_per_update.png", dpi=300, bbox_inches="tight")
    plt.show()

    # =============================================================================
    # 11. Save the Trained XLSTM Model and Performance Metrics
    # =============================================================================


    os.chdir('/kaggle/working/')
    model_save_path = "xlstm_model_incremental.pt"
    torch.save(modelxlstm.state_dict(), model_save_path)
    print(f"XLSTM model saved to {model_save_path}")
    os.chdir('/kaggle/working/')
    np.savez("xlstm_metrics.npz",
             incremental_update=np.array(metrics_history["incremental_update"]),
             avg_loss=np.array(metrics_history["avg_loss"]),
             avg_val_loss=np.array(metrics_history["avg_val_loss"]),
             avg_accuracy=np.array(metrics_history["avg_accuracy"]),
             avg_val_accuracy=np.array(metrics_history["avg_val_accuracy"]),
             update_time=np.array(metrics_history["update_time"]),
             forecast_pred=np.array(predicted_values),
             actual=np.array(actual_values))
    print("XLSTM model metrics saved to xlstm_metrics.npz")



  

# Comparison of all models

In [ ]:
#os.chdir('/kaggle/working/')


In [ ]:
#os.chdir('/kaggle/working/')
#import os
print(os.getcwd())


In [ ]:
"""import numpy as np
import matplotlib.pyplot as plt

# ---------------------------
# Load Metrics for Each Model
# ---------------------------
hybrid_data = np.load("hybrid_metrics.npz")
cnn_data    = np.load("cnn_metrics.npz")
lstm_data   = np.load("lstm_metrics.npz")
xlstm_data  = np.load("xlstm_metrics.npz")

# For Hybrid, CNN, and incremental LSTM, use:
#  - incremental_update: global update number
#  - avg_loss and avg_val_loss: training/validation loss
#  - avg_accuracy and avg_val_accuracy: training/validation accuracy
#  - update_time: training time per update
#  - forecast_pred and actual: forecast predictions and ground truth

# Extract variables for the Hybrid model
hybrid_updates       = hybrid_data["incremental_update"]
hybrid_loss          = hybrid_data["avg_loss"]
hybrid_val_loss      = hybrid_data["avg_val_loss"]
hybrid_accuracy      = hybrid_data["avg_accuracy"]
hybrid_val_accuracy  = hybrid_data["avg_val_accuracy"]
hybrid_update_times  = hybrid_data["update_time"]
hybrid_forecast      = hybrid_data["forecast_pred"]
hybrid_actual        = hybrid_data["actual"]

# Extract variables for the CNN model
cnn_updates       = cnn_data["incremental_update"]
cnn_loss          = cnn_data["avg_loss"]
cnn_val_loss      = cnn_data["avg_val_loss"]
cnn_accuracy      = cnn_data["avg_accuracy"]
cnn_val_accuracy  = cnn_data["avg_val_accuracy"]
cnn_update_times  = cnn_data["update_time"]
cnn_forecast      = cnn_data["forecast_pred"]
cnn_actual        = cnn_data["actual"]

# Extract variables for the Incremental LSTM model
lstm_updates       = lstm_data["incremental_update"]
lstm_loss          = lstm_data["avg_loss"]
lstm_val_loss      = lstm_data["avg_val_loss"]
lstm_accuracy      = lstm_data["avg_accuracy"]
lstm_val_accuracy  = lstm_data["avg_val_accuracy"]
lstm_update_times  = lstm_data["update_time"]
lstm_forecast      = lstm_data["forecast_pred"]
lstm_actual        = lstm_data["actual"]

# Extract variables for the XLSTM model (using the same keys)
xlstm_updates       = xlstm_data["incremental_update"]
xlstm_loss          = xlstm_data["avg_loss"]
xlstm_val_loss      = xlstm_data["avg_val_loss"]
xlstm_accuracy      = xlstm_data["avg_accuracy"]
xlstm_val_accuracy  = xlstm_data["avg_val_accuracy"]
xlstm_update_times  = xlstm_data["update_time"]
xlstm_forecast      = xlstm_data["forecast_pred"]
xlstm_actual        = xlstm_data["actual"]

# For forecasting, we assume that the actual forecast target is identical across models.
actual_forecast = hybrid_actual  # shape: (48,)

# ---------------------------
# Create Subplots for Comparison
# ---------------------------
fig, axs = plt.subplots(2, 2, figsize=(16, 14))

# --- Subplot 1: Training and Validation Loss vs. Incremental Update ---
axs[0, 0].plot(hybrid_updates, hybrid_loss, label='iTBG-Net Train Loss', color='blue')
axs[0, 0].plot(hybrid_updates, hybrid_val_loss, label='iTBG-Net Val Loss', color='blue', linestyle='--')
axs[0, 0].plot(cnn_updates, cnn_loss, label='CNN Train Loss', color='green')
axs[0, 0].plot(cnn_updates, cnn_val_loss, label='CNN Val Loss', color='green', linestyle='--')
axs[0, 0].plot(lstm_updates, lstm_loss, label='LSTM Train Loss', color='red')
axs[0, 0].plot(lstm_updates, lstm_val_loss, label='LSTM Val Loss', color='red', linestyle='--')
axs[0, 0].plot(xlstm_updates, xlstm_loss, label='XLSTM Train Loss', color='purple')
axs[0, 0].plot(xlstm_updates, xlstm_val_loss, label='XLSTM Val Loss', color='purple', linestyle='--')

axs[0, 0].set_title('Training & Validation Loss')
axs[0, 0].set_xlabel('Incremental Update')
axs[0, 0].set_ylabel('Loss')
axs[0, 0].legend()
axs[0, 0].grid(True, linestyle='--', alpha=0.6)

# --- Subplot 2: Training and Validation Accuracy vs. Incremental Update ---
axs[0, 1].plot(hybrid_updates, hybrid_accuracy, label='iTBG-Net Train Accuracy', color='blue')
axs[0, 1].plot(hybrid_updates, hybrid_val_accuracy, label='iTBG-Net Val Accuracy', color='blue', linestyle='--')
axs[0, 1].plot(cnn_updates, cnn_accuracy, label='CNN Train Accuracy', color='green')
axs[0, 1].plot(cnn_updates, cnn_val_accuracy, label='CNN Val Accuracy', color='green', linestyle='--')
axs[0, 1].plot(lstm_updates, lstm_accuracy, label='LSTM Train Accuracy', color='red')
axs[0, 1].plot(lstm_updates, lstm_val_accuracy, label='LSTM Val Accuracy', color='red', linestyle='--')
axs[0, 1].plot(xlstm_updates, xlstm_accuracy, label='XLSTM Train Accuracy', color='purple')
axs[0, 1].plot(xlstm_updates, xlstm_val_accuracy, label='XLSTM Val Accuracy', color='purple', linestyle='--')

axs[0, 1].set_title('Training & Validation Accuracy')
axs[0, 1].set_xlabel('Incremental Update')
axs[0, 1].set_ylabel('Regression Accuracy')
axs[0, 1].legend()
axs[0, 1].grid(True, linestyle='--', alpha=0.6)

# --- Subplot 3: Training Time per Incremental Update ---
axs[1, 0].plot(hybrid_updates, hybrid_update_times, label='iTBG-Net', color='blue')
axs[1, 0].plot(cnn_updates, cnn_update_times, label='CNN', color='green')
axs[1, 0].plot(lstm_updates, lstm_update_times, label='LSTM', color='red')
axs[1, 0].plot(xlstm_updates, xlstm_update_times, label='XLSTM', color='purple')

axs[1, 0].set_title('Training Time per Incremental Update')
axs[1, 0].set_xlabel('Incremental Update')
axs[1, 0].set_ylabel('Time (seconds)')
axs[1, 0].legend()
axs[1, 0].grid(True, linestyle='--', alpha=0.6)

# --- Subplot 4: Forecasting Comparison (48-step) ---
time_steps = np.arange(48)
axs[1, 1].plot(time_steps, actual_forecast, label='Actual', color='black', marker='o')
axs[1, 1].plot(time_steps, hybrid_forecast, label='iTBG-Net Forecast', color='blue', linestyle='--', marker='s')
axs[1, 1].plot(time_steps, cnn_forecast, label='CNN Forecast', color='green', linestyle='--', marker='s')
axs[1, 1].plot(time_steps, lstm_forecast, label='LSTM Forecast', color='red', linestyle='--', marker='s')
axs[1, 1].plot(time_steps, xlstm_forecast, label='XLSTM Forecast', color='purple', linestyle='--', marker='s')

axs[1, 1].set_title('48-step Forecasting Comparison')
axs[1, 1].set_xlabel('Time Step')
axs[1, 1].set_ylabel('Energy Demand')
axs[1, 1].legend()
axs[1, 1].grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig("model_comparison.png")
plt.show()
"""

The above results show that after some time, incremental learning keeps learning and batch-based learning stops learning, and after a while it keeps the lines straight, which means the model doesn't have more capacity to learn new data points, but the streaming model also keeps the old training and learning new data points, which is a good gesture to the natural learning process.

# Comparison of Models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Load Metrics for Each Model
# -----------------------------
hybrid_data = np.load("hybrid_metrics.npz")
cnn_data    = np.load("cnn_metrics.npz")
lstm_data   = np.load("lstm_metrics.npz")
xlstm_data  = np.load("xlstm_metrics.npz")

# For Hybrid, CNN, and incremental LSTM, the keys are:
#   incremental_update, avg_loss, avg_val_loss, avg_accuracy, avg_val_accuracy, update_time, forecast_pred, actual

# Extract variables for the Hybrid model.
hybrid_updates       = hybrid_data["incremental_update"]
hybrid_loss          = hybrid_data["avg_loss"]
hybrid_val_loss      = hybrid_data["avg_val_loss"]
hybrid_accuracy      = hybrid_data["avg_accuracy"]
hybrid_val_accuracy  = hybrid_data["avg_val_accuracy"]
hybrid_update_times  = hybrid_data["update_time"]
hybrid_forecast      = hybrid_data["forecast_pred"]
hybrid_actual        = hybrid_data["actual"]

# Extract variables for the CNN model.
cnn_updates       = cnn_data["incremental_update"]
cnn_loss          = cnn_data["avg_loss"]
cnn_val_loss      = cnn_data["avg_val_loss"]
cnn_accuracy      = cnn_data["avg_accuracy"]
cnn_val_accuracy  = cnn_data["avg_val_accuracy"]
cnn_update_times  = cnn_data["update_time"]
cnn_forecast      = cnn_data["forecast_pred"]
cnn_actual        = cnn_data["actual"]

# Extract variables for the incremental LSTM model.
lstm_updates       = lstm_data["incremental_update"]
lstm_loss          = lstm_data["avg_loss"]
lstm_val_loss      = lstm_data["avg_val_loss"]
lstm_accuracy      = lstm_data["avg_accuracy"]
lstm_val_accuracy  = lstm_data["avg_val_accuracy"]
lstm_update_times  = lstm_data["update_time"]
lstm_forecast      = lstm_data["forecast_pred"]
lstm_actual        = lstm_data["actual"]

# For the xLSTM model, we now also use the same keys.
xlstm_updates       = xlstm_data["incremental_update"]
xlstm_loss          = xlstm_data["avg_loss"]
xlstm_val_loss      = xlstm_data["avg_val_loss"]
xlstm_accuracy      = xlstm_data["avg_accuracy"]
xlstm_val_accuracy  = xlstm_data["avg_val_accuracy"]
xlstm_update_times  = xlstm_data["update_time"]
xlstm_forecast      = xlstm_data["forecast_pred"]
xlstm_actual        = xlstm_data["actual"]

# For forecasting, assume that the actual target is identical across models.
actual_forecast = hybrid_actual  # shape: (48,)

# -----------------------------
# Create Subplots for Comparison
# -----------------------------
fig, axs = plt.subplots(2, 2, figsize=(16, 14))

# --- Subplot 1: Training and Validation Loss vs. Incremental Update ---
axs[0, 0].plot(hybrid_updates, hybrid_loss, label='iTBG-Net Train Loss', color='blue')
axs[0, 0].plot(hybrid_updates, hybrid_val_loss, label='iTBG-Net Val Loss', color='blue', linestyle='--')
axs[0, 0].plot(cnn_updates, cnn_loss, label='CNN Train Loss', color='green')
axs[0, 0].plot(cnn_updates, cnn_val_loss, label='CNN Val Loss', color='green', linestyle='--')
axs[0, 0].plot(lstm_updates, lstm_loss, label='LSTM Train Loss', color='red')
axs[0, 0].plot(lstm_updates, lstm_val_loss, label='LSTM Val Loss', color='red', linestyle='--')
axs[0, 0].plot(xlstm_updates, xlstm_loss, label='XLSTM Train Loss', color='purple')
axs[0, 0].plot(xlstm_updates, xlstm_val_loss, label='XLSTM Val Loss', color='purple', linestyle='--')
axs[0, 0].set_title('Training & Validation Loss')
axs[0, 0].set_xlabel('Incremental Update')
axs[0, 0].set_ylabel('Loss')
axs[0, 0].legend(fontsize=9)
axs[0, 0].grid(True, linestyle='--', alpha=0.6)

# --- Subplot 2: Training and Validation Accuracy vs. Incremental Update ---
axs[0, 1].plot(hybrid_updates, hybrid_accuracy, label='iTBG-Net Train Accuracy', color='blue')
axs[0, 1].plot(hybrid_updates, hybrid_val_accuracy, label='iTBG-Net Val Accuracy', color='blue', linestyle='--')
axs[0, 1].plot(cnn_updates, cnn_accuracy, label='CNN Train Accuracy', color='green')
axs[0, 1].plot(cnn_updates, cnn_val_accuracy, label='CNN Val Accuracy', color='green', linestyle='--')
axs[0, 1].plot(lstm_updates, lstm_accuracy, label='LSTM Train Accuracy', color='red')
axs[0, 1].plot(lstm_updates, lstm_val_accuracy, label='LSTM Val Accuracy', color='red', linestyle='--')
axs[0, 1].plot(xlstm_updates, xlstm_accuracy, label='XLSTM Train Accuracy', color='purple')
axs[0, 1].plot(xlstm_updates, xlstm_val_accuracy, label='XLSTM Val Accuracy', color='purple', linestyle='--')
axs[0, 1].set_title('Training & Validation Accuracy')
axs[0, 1].set_xlabel('Incremental Update')
axs[0, 1].set_ylabel('Regression Accuracy')
axs[0, 1].legend(fontsize=9)
axs[0, 1].grid(True, linestyle='--', alpha=0.6)

# --- Subplot 3: Training Time per Incremental Update ---
axs[1, 0].plot(hybrid_updates, hybrid_update_times, label='iTBG-Net', color='blue')
axs[1, 0].plot(cnn_updates, cnn_update_times, label='CNN', color='green')
axs[1, 0].plot(lstm_updates, lstm_update_times, label='LSTM', color='red')
axs[1, 0].plot(xlstm_updates, xlstm_update_times, label='XLSTM', color='purple')
axs[1, 0].set_title('Training Time per Incremental Update')
axs[1, 0].set_xlabel('Incremental Update')
axs[1, 0].set_ylabel('Time (seconds)')
axs[1, 0].legend(fontsize=9)
axs[1, 0].grid(True, linestyle='--', alpha=0.6)

# --- Subplot 4: 48-step Forecasting Comparison ---
time_steps = np.arange(48)
axs[1, 1].plot(time_steps, actual_forecast, label='Actual', color='black', marker='o')
axs[1, 1].plot(time_steps, hybrid_forecast, label='iTBG-Net Forecast', color='blue', linestyle='--', marker='s')
axs[1, 1].plot(time_steps, cnn_forecast, label='CNN Forecast', color='green', linestyle='--', marker='s')
axs[1, 1].plot(time_steps, lstm_forecast, label='LSTM Forecast', color='red', linestyle='--', marker='s')
axs[1, 1].plot(time_steps, xlstm_forecast, label='XLSTM Forecast', color='purple', linestyle='--', marker='s')
axs[1, 1].set_title('48-step Forecasting Comparison')
axs[1, 1].set_xlabel('Time Step')
axs[1, 1].set_ylabel('Energy Demand (kWh)')
axs[1, 1].legend(fontsize=9)
axs[1, 1].grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig("model_comparison.png")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from IPython.display import FileLink, display

# -----------------------------
# Load Metrics for Each Model
# -----------------------------
hybrid_data = np.load("hybrid_metrics.npz")
cnn_data    = np.load("cnn_metrics.npz")
lstm_data   = np.load("lstm_metrics.npz")
xlstm_data  = np.load("xlstm_metrics.npz")

# For Hybrid, CNN, and incremental LSTM models, use the following keys:
#   - incremental_update: global update number
#   - avg_loss / avg_val_loss: training/validation loss
#   - avg_accuracy / avg_val_accuracy: training/validation accuracy
#   - update_time: training time per update
#   - forecast_pred and actual: forecast predictions and ground truth

# Extract arrays for the Hybrid model.
hybrid_updates       = hybrid_data["incremental_update"]
hybrid_loss          = hybrid_data["avg_loss"]
hybrid_val_loss      = hybrid_data["avg_val_loss"]
hybrid_accuracy      = hybrid_data["avg_accuracy"]
hybrid_val_accuracy  = hybrid_data["avg_val_accuracy"]
hybrid_update_times  = hybrid_data["update_time"]
hybrid_forecast      = hybrid_data["forecast_pred"]
hybrid_actual        = hybrid_data["actual"]

# Extract arrays for the CNN model.
cnn_updates       = cnn_data["incremental_update"]
cnn_loss          = cnn_data["avg_loss"]
cnn_val_loss      = cnn_data["avg_val_loss"]
cnn_accuracy      = cnn_data["avg_accuracy"]
cnn_val_accuracy  = cnn_data["avg_val_accuracy"]
cnn_update_times  = cnn_data["update_time"]
cnn_forecast      = cnn_data["forecast_pred"]
cnn_actual        = cnn_data["actual"]

# Extract arrays for the Incremental LSTM model.
lstm_updates       = lstm_data["incremental_update"]
lstm_loss          = lstm_data["avg_loss"]
lstm_val_loss      = lstm_data["avg_val_loss"]
lstm_accuracy      = lstm_data["avg_accuracy"]
lstm_val_accuracy  = lstm_data["avg_val_accuracy"]
lstm_update_times  = lstm_data["update_time"]
lstm_forecast      = lstm_data["forecast_pred"]
lstm_actual        = lstm_data["actual"]

# Extract arrays for the xLSTM model.
xlstm_updates       = xlstm_data["incremental_update"]
xlstm_loss          = xlstm_data["avg_loss"]
xlstm_val_loss      = xlstm_data["avg_val_loss"]
xlstm_accuracy      = xlstm_data["avg_accuracy"]
xlstm_val_accuracy  = xlstm_data["avg_val_accuracy"]
xlstm_update_times  = xlstm_data["update_time"]
xlstm_forecast      = xlstm_data["forecast_pred"]
xlstm_actual        = xlstm_data["actual"]

# For forecasting, assume that the actual target is identical across models.
actual_forecast = hybrid_actual  # shape: (48,)

dpi_val = 300  # DPI for saved images

# -----------------------------
# Plot 1: Training Loss vs. Incremental Update (use '*' marker)
# -----------------------------
plt.figure(figsize=(6, 4))
plt.plot(hybrid_updates, hybrid_loss, label='iTBG-Net Train Loss', color='blue', marker='*', linewidth=1)
plt.plot(cnn_updates, cnn_loss, label='CNN (IL) Train Loss', color='green', marker='*', linewidth=1)
plt.plot(lstm_updates, lstm_loss, label='LSTM (IL) Train Loss', color='red', marker='*', linewidth=1)
plt.plot(xlstm_updates, xlstm_loss, label='XLSTM (IL) Train Loss', color='purple', marker='*', linewidth=1)
plt.xlabel('Incremental Updates', fontweight='bold')
plt.ylabel('Loss', fontweight='bold')
plt.legend(fontsize=9, prop={'weight': 'bold'})
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.tight_layout()
plt.savefig("Comp_training_loss.eps", dpi=dpi_val)
plt.savefig("Comp_training_loss.pdf", dpi=dpi_val)
plt.savefig("Comp_training_loss.png", dpi=dpi_val)
plt.show()

# -----------------------------
# Plot 2: Validation Loss vs. Incremental Update (use '.' marker)
# -----------------------------
plt.figure(figsize=(6, 4))
plt.plot(hybrid_updates, hybrid_val_loss, label='iTBG-Net Val Loss', color='blue', linestyle='--', marker='.', linewidth=1)
plt.plot(cnn_updates, cnn_val_loss, label='CNN (IL) Val Loss', color='green', linestyle='--', marker='.', linewidth=1)
plt.plot(lstm_updates, lstm_val_loss, label='LSTM (IL) Val Loss', color='red', linestyle='--', marker='.', linewidth=1)
plt.plot(xlstm_updates, xlstm_val_loss, label='XLSTM (IL) Val Loss', color='purple', linestyle='--', marker='.', linewidth=1)
plt.xlabel('Incremental Updates', fontweight='bold')
plt.ylabel('Loss', fontweight='bold')
plt.legend(fontsize=9, prop={'weight': 'bold'})
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.tight_layout()
plt.savefig("Comp_validation_loss.eps", dpi=dpi_val)
plt.savefig("Comp_validation_loss.pdf", dpi=dpi_val)
plt.savefig("Comp_validation_loss.png", dpi=dpi_val)
plt.show()

# -----------------------------
# Plot 3: Training Accuracy vs. Incremental Update (use '*' marker)
# -----------------------------
plt.figure(figsize=(6, 4))
plt.plot(hybrid_updates, hybrid_accuracy, label='iTBG-Net Train Acc', color='blue', marker='*', linewidth=1)
plt.plot(cnn_updates, cnn_accuracy, label='CNN (IL) Train Acc', color='green', marker='*', linewidth=1)
plt.plot(lstm_updates, lstm_accuracy, label='LSTM (IL) Train Acc', color='red', marker='*', linewidth=1)
plt.plot(xlstm_updates, xlstm_accuracy, label='XLSTM (IL) Train Acc', color='purple', marker='*', linewidth=1)
plt.xlabel('Incremental Updates', fontweight='bold')
plt.ylabel('Regression Accuracy', fontweight='bold')
plt.legend(fontsize=9, prop={'weight': 'bold'})
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.tight_layout()
plt.savefig("Comp_training_accuracy.eps", dpi=dpi_val)
plt.savefig("Comp_training_accuracy.pdf", dpi=dpi_val)
plt.savefig("Comp_training_accuracy.png", dpi=dpi_val)
plt.show()

# -----------------------------
# Plot 4: Validation Accuracy vs. Incremental Update (use '.' marker)
# -----------------------------
plt.figure(figsize=(6, 4))
plt.plot(hybrid_updates, hybrid_val_accuracy, label='iTBG-Net Val Acc', color='blue', linestyle='--', marker='.', linewidth=1)
plt.plot(cnn_updates, cnn_val_accuracy, label='CNN (IL) Val Acc', color='green', linestyle='--', marker='.', linewidth=1)
plt.plot(lstm_updates, lstm_val_accuracy, label='LSTM (IL) Val Acc', color='red', linestyle='--', marker='.', linewidth=1)
plt.plot(xlstm_updates, xlstm_val_accuracy, label='XLSTM (IL) Val Acc', color='purple', linestyle='--', marker='.', linewidth=1)
plt.xlabel('Incremental Update', fontweight='bold')
plt.ylabel('Regression Accuracy', fontweight='bold')
plt.legend(fontsize=9, prop={'weight': 'bold'})
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.tight_layout()
plt.savefig("Comp_validation_accuracy.eps", dpi=dpi_val)
plt.savefig("Comp_validation_accuracy.pdf", dpi=dpi_val)
plt.savefig("Comp_validation_accuracy.png", dpi=dpi_val)
plt.show()

# -----------------------------
# Plot 5: Training Time per Incremental Update (with enhanced circle markers)
# -----------------------------
plt.figure(figsize=(6, 4))
plt.plot(hybrid_updates, hybrid_update_times, label='iTBG-Net', marker='o', linestyle='-', markersize=5, linewidth=2,
         color='blue', markerfacecolor='white', markeredgewidth=1.2, markeredgecolor='blue')
plt.plot(cnn_updates, cnn_update_times, label='CNN (IL)', marker='o', linestyle='-', markersize=5, linewidth=2,
         color='green', markerfacecolor='white', markeredgewidth=1.2, markeredgecolor='green')
plt.plot(lstm_updates, lstm_update_times, label='LSTM (IL)', marker='o', linestyle='-', markersize=5, linewidth=2,
         color='red', markerfacecolor='white', markeredgewidth=1.2, markeredgecolor='red')
plt.plot(xlstm_updates, xlstm_update_times, label='XLSTM (IL)', marker='o', linestyle='-', markersize=5, linewidth=2,
         color='purple', markerfacecolor='white', markeredgewidth=1.2, markeredgecolor='purple')
plt.xlabel('Incremental Updates', fontweight='bold')
plt.ylabel('Time (seconds)', fontweight='bold')
plt.legend(fontsize=9, prop={'weight': 'bold'})
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.tight_layout()
plt.savefig("Comp_training_time.eps", dpi=dpi_val)
plt.savefig("Comp_training_time.pdf", dpi=dpi_val)
plt.savefig("Comp_training_time.png", dpi=dpi_val)
plt.show()

# -----------------------------
# Plot 6: Forecasting Comparison (48-step) with updated markers
# -----------------------------
plt.figure(figsize=(6, 4))
time_steps = np.arange(48)
# "Actual" uses a star marker and firebrick color.
plt.plot(time_steps, actual_forecast, label='Actual', color='firebrick', marker='*', linewidth=1)
# Forecast curves use a dot marker and dashed linestyle.
plt.plot(time_steps, hybrid_forecast, label='iTBG-Net Forecast', color='blue', linestyle='--', marker='.', linewidth=1)
plt.plot(time_steps, cnn_forecast, label='CNN (IL) Forecast', color='green', linestyle='--', marker='.', linewidth=1)
plt.plot(time_steps, lstm_forecast, label='LSTM (IL) Forecast', color='red', linestyle='--', marker='.', linewidth=1)
plt.plot(time_steps, xlstm_forecast, label='XLSTM (IL) Forecast', color='purple', linestyle='--', marker='.', linewidth=1)
plt.xlabel('Time Step', fontweight='bold')
plt.ylabel('Energy Demand (kWh)', fontweight='bold')
plt.legend(fontsize=9, prop={'weight': 'bold'})
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.tight_layout()
plt.savefig("Comp_forecasting_comparison.eps", dpi=dpi_val)
plt.savefig("Comp_forecasting_comparison.pdf", dpi=dpi_val)
plt.savefig("Comp_forecasting_comparison.png", dpi=dpi_val)
plt.show()

# -----------------------------
# Zip all saved plot files and display a download link
# -----------------------------
file_list = [
    "Comp_training_loss.eps", "Comp_training_loss.pdf", "Comp_training_loss.png",
    "Comp_validation_loss.eps", "Comp_validation_loss.pdf", "Comp_validation_loss.png",
    "Comp_training_accuracy.eps", "Comp_training_accuracy.pdf", "Comp_training_accuracy.png",
    "Comp_validation_accuracy.eps", "Comp_validation_accuracy.pdf", "Comp_validation_accuracy.png",
    "Comp_training_time.eps", "Comp_training_time.pdf", "Comp_training_time.png",
    "Comp_forecasting_comparison.eps", "Comp_forecasting_comparison.pdf", "Comp_forecasting_comparison.png"
]

zip_filename = "plots.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for filename in file_list:
        zipf.write(filename)

# Display the download link in the notebook.
display(FileLink(zip_filename))


In [ ]:
!pip install dataframe_image

In [ ]:
!pip install nest_asyncio

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from IPython.display import FileLink, display

# -----------------------------
# Load Metrics for Each Model (using absolute paths)
# -----------------------------
hybrid_data     = np.load("/kaggle/working/hybrid_metrics.npz")
cnn_data        = np.load("/kaggle/working/cnn_metrics.npz")
lstm_data       = np.load("/kaggle/working/lstm_metrics.npz")
xlstm_data      = np.load("/kaggle/working/xlstm_metrics.npz")

# For Hybrid, CNN, and incremental LSTM models, use these keys:
#   incremental_update, avg_loss, avg_val_loss, avg_accuracy, avg_val_accuracy, update_time, forecast_pred, actual

# Extract variables for the Hybrid model.
hybrid_updates       = hybrid_data["incremental_update"]
hybrid_loss          = hybrid_data["avg_loss"]
hybrid_val_loss      = hybrid_data["avg_val_loss"]
hybrid_accuracy      = hybrid_data["avg_accuracy"]
hybrid_val_accuracy  = hybrid_data["avg_val_accuracy"]
hybrid_epoch_times   = hybrid_data["update_time"]
hybrid_forecast      = hybrid_data["forecast_pred"]
hybrid_actual        = hybrid_data["actual"]

# Extract variables for the CNN model.
cnn_updates       = cnn_data["incremental_update"]
cnn_loss          = cnn_data["avg_loss"]
cnn_val_loss      = cnn_data["avg_val_loss"]
cnn_accuracy      = cnn_data["avg_accuracy"]
cnn_val_accuracy  = cnn_data["avg_val_accuracy"]
cnn_epoch_times   = cnn_data["update_time"]
cnn_forecast      = cnn_data["forecast_pred"]
cnn_actual        = cnn_data["actual"]

# Extract variables for the Incremental LSTM model.
lstm_updates       = lstm_data["incremental_update"]
lstm_loss          = lstm_data["avg_loss"]
lstm_val_loss      = lstm_data["avg_val_loss"]
lstm_accuracy      = lstm_data["avg_accuracy"]
lstm_val_accuracy  = lstm_data["avg_val_accuracy"]
lstm_epoch_times   = lstm_data["update_time"]
lstm_forecast      = lstm_data["forecast_pred"]
lstm_actual        = lstm_data["actual"]

# Extract variables for the xLSTM model.
xlstm_updates       = xlstm_data["incremental_update"]
xlstm_loss          = xlstm_data["avg_loss"]
xlstm_val_loss      = xlstm_data["avg_val_loss"]
xlstm_accuracy      = xlstm_data["avg_accuracy"]
xlstm_val_accuracy  = xlstm_data["avg_val_accuracy"]
xlstm_epoch_times   = xlstm_data["update_time"]
xlstm_forecast      = xlstm_data["forecast_pred"]
xlstm_actual        = xlstm_data["actual"]

# For forecasting, assume the actual target is identical across models.
# (Here we use Hybrid's "actual".)
actual_forecast = hybrid_actual  # shape: (48,)

dpi_val = 300  # DPI for high-quality saved images

# -----------------------------
# Compute Mean Metrics for Each Model
# -----------------------------
def forecast_mae(forecast, actual):
    return np.mean(np.abs(forecast - actual))

hybrid_mean_train_loss = np.mean(hybrid_loss)
hybrid_mean_val_loss   = np.mean(hybrid_val_loss)
hybrid_mean_train_acc  = np.mean(hybrid_accuracy)
hybrid_mean_val_acc    = np.mean(hybrid_val_accuracy)
hybrid_mean_time       = np.mean(hybrid_epoch_times)
hybrid_forecast_mae    = forecast_mae(hybrid_forecast, hybrid_actual)

cnn_mean_train_loss = np.mean(cnn_loss)
cnn_mean_val_loss   = np.mean(cnn_val_loss)
cnn_mean_train_acc  = np.mean(cnn_accuracy)
cnn_mean_val_acc    = np.mean(cnn_val_accuracy)
cnn_mean_time       = np.mean(cnn_epoch_times)
cnn_forecast_mae    = forecast_mae(cnn_forecast, cnn_actual)

lstm_mean_train_loss = np.mean(lstm_loss)
lstm_mean_val_loss   = np.mean(lstm_val_loss)
lstm_mean_train_acc  = np.mean(lstm_accuracy)
lstm_mean_val_acc    = np.mean(lstm_val_accuracy)
lstm_mean_time       = np.mean(lstm_epoch_times)
lstm_forecast_mae    = forecast_mae(lstm_forecast, lstm_actual)

xlstm_mean_train_loss = np.mean(xlstm_loss)
xlstm_mean_val_loss   = np.mean(xlstm_val_loss)
xlstm_mean_train_acc  = np.mean(xlstm_accuracy)
xlstm_mean_val_acc    = np.mean(xlstm_val_accuracy)
xlstm_mean_time       = np.mean(xlstm_epoch_times)
xlstm_forecast_mae    = forecast_mae(xlstm_forecast, xlstm_actual)

# -----------------------------
# Create a Comparison DataFrame
# -----------------------------
df = pd.DataFrame({
    "Model": ["Proposed iTBG-Net", "CNN", "LSTM Incremental", "XLSTM"],
    "Train Loss": [hybrid_mean_train_loss, cnn_mean_train_loss, lstm_mean_train_loss, xlstm_mean_train_loss],
    "Val Loss": [hybrid_mean_val_loss, cnn_mean_val_loss, lstm_mean_val_loss, xlstm_mean_val_loss],
    "Train Acc": [hybrid_mean_train_acc, cnn_mean_train_acc, lstm_mean_train_acc, xlstm_mean_train_acc],
    "Val Acc": [hybrid_mean_val_acc, cnn_mean_val_acc, lstm_mean_val_acc, xlstm_mean_val_acc],
    "Update Time (s)": [hybrid_mean_time, cnn_mean_time, lstm_mean_time, xlstm_mean_time],
    "Forecast MAE": [hybrid_forecast_mae, cnn_forecast_mae, lstm_forecast_mae, xlstm_forecast_mae]
})

# -----------------------------
# Highlight Best Values Using LaTeX Bold Formatting
# -----------------------------
# For metrics where lower is better (e.g., Loss, Update Time, Forecast MAE), we highlight the lowest;
# for those where higher is better (e.g., Accuracy), we highlight the highest.
def format_cell(val, best, higher_is_better, decimals=6):
    fmt_val = f"{val:.{decimals}f}"
    if higher_is_better:
        if np.isclose(val, best, atol=1e-6):
            return r"$\mathbf{" + fmt_val + "}$"
    else:
        if np.isclose(val, best, atol=1e-6):
            return r"$\mathbf{" + fmt_val + "}$"
    return fmt_val

df_formatted = df.copy()
for col in df.columns:
    if col == "Model":
        continue
    if col in ["Train Acc", "Val Acc"]:
        best = df[col].max()
        df_formatted[col] = df[col].apply(lambda x: format_cell(x, best, higher_is_better=True))
    else:
        best = df[col].min()
        df_formatted[col] = df[col].apply(lambda x: format_cell(x, best, higher_is_better=False))

# Create a 2D list for the table including headers.
table_data = [df_formatted.columns.tolist()] + df_formatted.values.tolist()

# -----------------------------
# Create a Matplotlib Table and Save as PNG and EPS
# -----------------------------
fig, ax = plt.subplots(figsize=(12, 3))
ax.axis('tight')
ax.axis('off')
the_table = ax.table(cellText=table_data, colLabels=None, loc='center', cellLoc='center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(10)
fig.tight_layout()
plt.savefig("/kaggle/working/model_comparison_table.png", dpi=300)
plt.savefig("/kaggle/working/model_comparison_table.eps", format='eps')
plt.show()

# Also print the plain DataFrame for reference.
print(df)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define model names and total training time (sum of update_time over all incremental updates)
models_training_time = {
    "iTBG-Net": np.sum(hybrid_update_times),
    "CNN (IL)": np.sum(cnn_update_times),
    "LSTM (IL)": np.sum(lstm_update_times),
    "XLSTM (IL)": np.sum(xlstm_update_times),
}

# Define modern colors for each model.
model_colors = {
    "iTBG-Net": "#1f77b4",   # Blue
    "CNN (IL)": "#2ca02c",   # Green
    "LSTM (IL)": "#d62728",  # Red
    "XLSTM (IL)": "#9467bd", # Purple
}

# Create a bar chart with high resolution
plt.figure(figsize=(6, 4), dpi=600)
bar_width = 0.5  # Set bar width

bars = plt.bar(
    list(models_training_time.keys()),
    list(models_training_time.values()),
    color=[model_colors[m] for m in models_training_time.keys()], 
    edgecolor='black', 
    linewidth=1.2, 
    width=bar_width, 
    alpha=0.85
)

# Add value labels on top of each bar, formatted with a thousands separator.
max_height = max(models_training_time.values())
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2, 
        height + 0.03 * max_height,  # Extra spacing above the bar
        f'{height:,.1f} sec', 
        ha='center', 
        va='bottom', 
        fontsize=10, 
        fontweight='bold', 
        color='black'
    )

# Adjust y-axis upper limit to leave extra space
plt.ylim(0, max_height * 1.15)

# Add axis labels and title
plt.ylabel("Training Time (seconds)", fontsize=10, fontweight="bold", labelpad=10)
plt.title("Training Time Comparison", fontsize=10, fontweight="bold", pad=10)

# Rotate x-axis labels for better readability and set them in bold.
plt.xticks(rotation=15, fontsize=10, fontweight="bold")

# Display grid only along y-axis
plt.grid(axis="y", linestyle="--", linewidth=0.6, alpha=0.7)

# Save the figure in high resolution
plt.savefig("Total_Training_Time_BarChart_all.eps", dpi=600, bbox_inches="tight")
plt.savefig("Total_Training_Time_BarChart_all.png", dpi=600, bbox_inches="tight")
plt.savefig("Total_Training_Time_BarChart_all.pdf", dpi=600, bbox_inches="tight")

# Show the plot
plt.show()


In [ ]:
"""import numpy as np
import matplotlib.pyplot as plt

# Define model names and forecasted values (Non-Incremental LSTM removed)
models = {
    "iTBG-Net": hybrid_forecast,
    "CNN (IL)": cnn_forecast,
    "LSTM (IL)": lstm_forecast,
    "XLSTM (IL)": xlstm_forecast,
}

# Define model colors
model_colors = {
    "iTBG-Net": "#1f77b4",    # Blue
    "CNN (IL)": "#2ca02c",    # Green
    "LSTM (IL)": "#d62728",   # Red
    "XLSTM (IL)": "#9467bd",  # Purple
}

# Compute Forecasting Accuracy for each model.
# Formula: 100 * (1 - (sum(|forecast - actual|) / sum(|actual|)))
def forecast_mae_percent(forecast, actual):
    return 100 * (1 - (np.sum(np.abs(actual - forecast)) / np.sum(np.abs(actual))))

accuracy_values = {}
for model_name, forecast in models.items():
    accuracy = forecast_mae_percent(forecast, actual_forecast)
    accuracy_values[model_name] = accuracy

# Create a high-resolution bar chart.
plt.figure(figsize=(6, 4), dpi=600)
bar_width = 0.5
bars = plt.bar(
    list(accuracy_values.keys()),
    list(accuracy_values.values()),
    color=[model_colors[m] for m in accuracy_values.keys()],
    edgecolor='black',
    linewidth=1.2,
    width=bar_width,
    alpha=0.85
)

# Add value labels on top of each bar (formatted to 2 decimal places).
max_height = max(accuracy_values.values())
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2,
        height + 0.02 * max_height,  # Extra spacing
        f'{height:.2f}%',
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold',
        color='black'
    )

plt.ylim(0, max_height * 1.15)  # Adding 15% extra space above the tallest bar
plt.ylabel("Forecasting Accuracy (%)", fontsize=10, fontweight='bold', labelpad=10)
plt.title("Accuracy Comparison of 48-Hour Forecasts", fontsize=10, fontweight='bold', pad=10)
plt.xticks(rotation=15, fontsize=10, fontweight="bold")
plt.grid(axis="y", linestyle='--', linewidth=0.6, alpha=0.7)
plt.savefig("Forecasting_Accuracy_BarChart.png", dpi=600, bbox_inches="tight")
plt.savefig("Forecasting_Accuracy_BarChart.eps", dpi=600, bbox_inches="tight")
plt.savefig("Forecasting_Accuracy_BarChart.pdf", dpi=600, bbox_inches="tight")
plt.show()
"""

In [ ]:
"""import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn

# ==============================================================================
# 1. Data Loading and Splitting into Training and Forecasting Sets
# ==============================================================================
# Load the dataset and sort by timestamp
data = pd.read_csv('/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.sort_values('Timestamp', inplace=True)

features = ['Energy_Demand']
sequence_length = 48  
prediction_length = 48  

# Use the last prediction_length (48) records as forecasting data.
n_total = len(data)
n_forecast = prediction_length          # Last 48 records
n_train = n_total - n_forecast         # Remaining records for training

training_data = data.iloc[:n_train].copy()
forecast_data = data.iloc[n_train:].copy()

print("Total samples in dataset:", n_total)
print("Forecasting samples (last 48):", n_forecast)
print("Training samples:", n_train)

# ==============================================================================
# 2. Mean-Based Scaling
# ==============================================================================
# Compute the training mean and scale both training and forecasting data accordingly.
training_mean = training_data[features].mean().values[0]
training_features = training_data[features].values / training_mean
forecast_features = forecast_data[features].values / training_mean

# ==============================================================================
# 3. Generate Overlapping Sequences from Training Data
# ==============================================================================
# Create sliding-window sequences and corresponding labels from training data.
training_sequences = []
training_labels = []
for i in range(len(training_features) - sequence_length - prediction_length + 1):
    seq = training_features[i : i + sequence_length]
    label = training_features[i + sequence_length : i + sequence_length + prediction_length]
    training_sequences.append(seq)
    training_labels.append(label)

training_sequences = np.array(training_sequences)         # Shape: (N, 48, 1)
training_labels = np.array(training_labels, dtype=np.float32)  # Shape: (N, 48, 1)

print("Total training sequences generated:", len(training_sequences))

# ==============================================================================
# 4. Define the Forecasting (Test) Set for Comparison
# ==============================================================================
# Define the test sequence as the last 48 hours of training_features.
# The corresponding ground truth (test_target) is the scaled forecast data.
test_sequence = training_features[-sequence_length:]  # Shape: (48, 1)
test_sequence = np.expand_dims(test_sequence, axis=0)   # Shape: (1, 48, 1)

# Ground truth for forecasting comes from forecast_data.
test_target = forecast_features.flatten()               # Shape: (48,)

# ==============================================================================
# 5. Custom Objects for Keras Model Loading (if needed)
# ==============================================================================
class GetItem(tf.keras.layers.Layer):
    def __init__(self, index=slice(None, None, None), **kwargs):
        super(GetItem, self).__init__(**kwargs)
        self.index = index

    def call(self, inputs):
        return inputs[self.index]

    def get_config(self):
        config = super(GetItem, self).get_config()
        config.update({"index": (self.index.start, self.index.stop, self.index.step)})
        return config

def r2(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - SS_res/(SS_tot + tf.keras.backend.epsilon())

def clipped_mape(y_true, y_pred):
    epsilon = 1e-3
    y_true_clipped = tf.maximum(y_true, epsilon)
    return tf.reduce_mean(tf.abs((y_true - y_pred)/y_true_clipped)) * 100

custom_objects = {
    "clipped_mape": clipped_mape,
    "r2": r2,
    "r2_metric": tf.keras.metrics.MeanMetricWrapper(r2, name="r2"),
    "GetItem": GetItem
}

# ==============================================================================
# 6. Model Prediction on the Forecasting Set
# ==============================================================================
# Assumption: These models have already been trained (or loaded) beforehand.
# Replace the following dummy variables with your actual model instances.
#
# For Keras models, we use .predict(), and for the PyTorch model, we forward the test tensor.
hybrid_pred = hybrid_model.predict(test_sequence)[0].flatten()       # Proposed iTBG-Net
cnn_pred = cnn_model.predict(test_sequence)[0].flatten()             # CNN incremental model
lstm_inc_pred = lstm_model.predict(test_sequence)[0].flatten()       # LSTM incremental model

# For the XLSTM incremental model (in PyTorch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelxlstm.to(device)
test_tensor = torch.tensor(test_sequence, dtype=torch.float32).to(device)  # Shape: (1, 48, 1)
modelxlstm.eval()
with torch.no_grad():
    xlstm_pred = modelxlstm(test_tensor).cpu().numpy().flatten()

# ==============================================================================
# 7. Compute Evaluation Metrics (MAE and MSE)
# ==============================================================================
def compute_metrics(forecast, actual):
    forecast = forecast.flatten()
    actual = actual.flatten()
    mae = np.mean(np.abs(forecast - actual))
    mse = np.mean((forecast - actual)**2)
    return mae, mse

hybrid_mae, hybrid_mse = compute_metrics(hybrid_pred, test_target)
cnn_mae, cnn_mse       = compute_metrics(cnn_pred, test_target)
lstm_inc_mae, lstm_inc_mse = compute_metrics(lstm_inc_pred, test_target)
xlstm_mae, xlstm_mse   = compute_metrics(xlstm_pred, test_target)

# ==============================================================================
# 8. Create a Comparison DataFrame and Print the Results
# ==============================================================================
df = pd.DataFrame({
    "Model": ["Proposed iTBG-Net", "CNN (Incremental)", "LSTM (Incremental)", "XLSTM (Incremental)"],
    "MAE": [hybrid_mae, cnn_mae, lstm_inc_mae, xlstm_mae],
    "MSE": [hybrid_mse, cnn_mse, lstm_inc_mse, xlstm_mse]
})
print(df)

# ==============================================================================
# 9. Visualization: Bar Plots for MAE and MSE on the Forecasting Set
# ==============================================================================
x = np.arange(len(df["Model"]))
bar_width = 0.5
colors = ["#1f77b4", "#2ca02c", "#d62728", "#9467bd"]

# MAE Comparison Plot
plt.figure(figsize=(6, 4), dpi=300)
plt.bar(x, df["MAE"], color=colors, width=bar_width, edgecolor="black")
plt.title("MAE Comparison on Forecasting Set", fontsize=10, fontweight="bold")
plt.ylabel("MAE", fontsize=10, fontweight="bold")
plt.xticks(x, df["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.savefig("mae_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

# MSE Comparison Plot
plt.figure(figsize=(6, 4), dpi=300)
plt.bar(x, df["MSE"], color=colors, width=bar_width, edgecolor="black")
plt.title("MSE Comparison on Forecasting Set", fontsize=10, fontweight="bold")
plt.ylabel("MSE", fontsize=10, fontweight="bold")
plt.xticks(x, df["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.savefig("mse_comparison.png", dpi=300, bbox_inches="tight")
plt.show()
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn

# ==============================================================================
# 1. Data Loading and Splitting (Only for Scaling Purposes)
# ==============================================================================
data = pd.read_csv('/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.sort_values('Timestamp', inplace=True)

features = ['Energy_Demand']
sequence_length = 48  
prediction_length = 48  

# Reserve the last 48 samples for forecasting.
n_total = len(data)
n_forecast = prediction_length          # Last 48 rows are used for forecasting.
n_train = n_total - n_forecast         # The remaining data is used only for computing the scaling parameter.

training_data = data.iloc[:n_train].copy()   # Used solely to compute the mean.
forecast_data = data.iloc[n_train:].copy()     # Ground truth for forecasting.

print("Total samples in dataset:", n_total)
print("Forecasting samples (last 48):", n_forecast)
print("Training samples (for scaling):", n_train)

# ==============================================================================
# 2. Mean-Based Scaling
# ==============================================================================
# Compute the training mean and scale both training and forecasting features accordingly.
training_mean = training_data[features].mean().values[0]
training_features = training_data[features].values / training_mean
forecast_features = forecast_data[features].values / training_mean

# ==============================================================================
# 3. Define Forecasting Input Sequence and Ground Truth Target
# ==============================================================================
# Use the last 48 hours from the scaled training features to form the forecasting input.
forecast_sequence = training_features[-sequence_length:]   # Shape: (48, 1)
forecast_sequence = np.expand_dims(forecast_sequence, axis=0)  # Shape: (1, 48, 1)

# The ground truth target is taken from the reserved forecast_data.
test_target = forecast_features.flatten()  # Shape: (48,)

# ==============================================================================
# 4. Custom Objects for Keras Model Loading (if needed)
# ==============================================================================
class GetItem(tf.keras.layers.Layer):
    def __init__(self, index=slice(None, None, None), **kwargs):
        super(GetItem, self).__init__(**kwargs)
        self.index = index

    def call(self, inputs):
        return inputs[self.index]

    def get_config(self):
        config = super(GetItem, self).get_config()
        config.update({"index": (self.index.start, self.index.stop, self.index.step)})
        return config

def r2(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - SS_res/(SS_tot + tf.keras.backend.epsilon())

def clipped_mape(y_true, y_pred):
    epsilon = 1e-3
    y_true_clipped = tf.maximum(y_true, epsilon)
    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true_clipped)) * 100

custom_objects = {
    "clipped_mape": clipped_mape,
    "r2": r2,
    "r2_metric": tf.keras.metrics.MeanMetricWrapper(r2, name="r2"),
    "GetItem": GetItem
}

# ==============================================================================
# 5. Model Prediction on the Forecasting Set
# ==============================================================================
# Assumption: The following models are already trained/loaded.
# Replace these dummy model instances with your actual model objects.
hybrid_pred = hybrid_model.predict(forecast_sequence)[0].flatten()       # Proposed iTBG-Net
cnn_pred = cnn_model.predict(forecast_sequence)[0].flatten()             # CNN (Incremental)
lstm_inc_pred = lstm_model.predict(forecast_sequence)[0].flatten()       # LSTM (Incremental)

# For the XLSTM incremental model (PyTorch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelxlstm.to(device)
test_tensor = torch.tensor(forecast_sequence, dtype=torch.float32).to(device)
modelxlstm.eval()
with torch.no_grad():
    xlstm_pred = modelxlstm(test_tensor).cpu().numpy().flatten()

# ==============================================================================
# 6. Compute Evaluation Metrics (MAE, MSE)
# ==============================================================================
def compute_metrics(forecast, actual):
    forecast = forecast.flatten()
    actual = actual.flatten()
    mae = np.mean(np.abs(forecast - actual))
    mse = np.mean((forecast - actual)**2)
    return mae, mse

hybrid_mae, hybrid_mse = compute_metrics(hybrid_pred, test_target)
cnn_mae, cnn_mse = compute_metrics(cnn_pred, test_target)
lstm_inc_mae, lstm_inc_mse = compute_metrics(lstm_inc_pred, test_target)
xlstm_mae, xlstm_mse = compute_metrics(xlstm_pred, test_target)

# Create a DataFrame to compare the MAE and MSE of the models.
df_metrics = pd.DataFrame({
    "Model": ["Proposed iTBG-Net", "CNN (Incremental)", "LSTM (Incremental)", "XLSTM (Incremental)"],
    "MAE": [hybrid_mae, cnn_mae, lstm_inc_mae, xlstm_mae],
    "MSE": [hybrid_mse, cnn_mse, lstm_inc_mse, xlstm_mse]
})
print(df_metrics)

# ==============================================================================
# 7. Visualization: Bar Plots for MAE and MSE on the Forecasting Set
# ==============================================================================
x = np.arange(len(df_metrics["Model"]))
bar_width = 0.5
colors = ["#1f77b4", "#2ca02c", "#d62728", "#9467bd"]

# MAE Bar Plot
plt.figure(figsize=(6, 4), dpi=300)
plt.bar(x, df_metrics["MAE"], color=colors, width=bar_width, edgecolor="black")
plt.title("MAE Comparison on Forecasting Set", fontsize=10, fontweight="bold")
plt.ylabel("MAE", fontsize=10, fontweight="bold")
plt.xticks(x, df_metrics["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.savefig("mae_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

# MSE Bar Plot
plt.figure(figsize=(6, 4), dpi=300)
plt.bar(x, df_metrics["MSE"], color=colors, width=bar_width, edgecolor="black")
plt.title("MSE Comparison on Forecasting Set", fontsize=10, fontweight="bold")
plt.ylabel("MSE", fontsize=10, fontweight="bold")
plt.xticks(x, df_metrics["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.savefig("mse_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

"""# ==============================================================================
# Accuracy Bar Plot for Model Comparison
# ==============================================================================
# Define a simple accuracy metric: percentage of predictions with error below a chosen tolerance.
def compute_accuracy(pred, actual, tol=0.1):
    # Returns the accuracy (%) where each prediction is considered accurate if its absolute error is less than tol.
    acc = np.mean(np.abs(pred - actual) < tol) * 100
    return acc

hybrid_acc = compute_accuracy(hybrid_pred, test_target)
cnn_acc = compute_accuracy(cnn_pred, test_target)
lstm_inc_acc = compute_accuracy(lstm_inc_pred, test_target)
xlstm_acc = compute_accuracy(xlstm_pred, test_target)

df_accuracy = pd.DataFrame({
    "Model": ["Proposed iTBG-Net", "CNN (Incremental)", "LSTM (Incremental)", "XLSTM (Incremental)"],
    "Accuracy (%)": [hybrid_acc, cnn_acc, lstm_inc_acc, xlstm_acc]
})
print(df_accuracy)

# Accuracy Bar Plot
plt.figure(figsize=(6, 4), dpi=300)
plt.bar(x, df_accuracy["Accuracy (%)"], color=colors, width=bar_width, edgecolor="black")
plt.title("Forecasting Accuracy Comparison", fontsize=10, fontweight="bold")
plt.ylabel("Accuracy (%)", fontsize=10, fontweight="bold")
plt.xticks(x, df_accuracy["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.ylim(0, 100)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.savefig("accuracy_comparison.png", dpi=300, bbox_inches="tight")
plt.show()
"""

In [ ]:
"""import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn

# ==============================================================================
# 1. Data Loading and Splitting (Only for Scaling Purposes)
# ==============================================================================
data = pd.read_csv('/kaggle/input/integrated-energy-management-and-forecasting/Integrated Energy Management and Forecasting Dataset.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.sort_values('Timestamp', inplace=True)

features = ['Energy_Demand']
sequence_length = 48  
prediction_length = 48  

# Reserve the last 48 samples for forecasting.
n_total = len(data)
n_forecast = prediction_length          # Last 48 rows used for forecasting.
n_train = n_total - n_forecast         # The remaining data is used only for computing the scaling parameter.

training_data = data.iloc[:n_train].copy()   # Used solely to compute the mean.
forecast_data = data.iloc[n_train:].copy()     # Ground truth for forecasting.

print("Total samples in dataset:", n_total)
print("Forecasting samples (last 48):", n_forecast)
print("Training samples (for scaling):", n_train)

# ==============================================================================
# 2. Mean-Based Scaling
# ==============================================================================
# Compute the training mean and scale both training and forecasting features accordingly.
training_mean = training_data[features].mean().values[0]
training_features = training_data[features].values / training_mean
forecast_features = forecast_data[features].values / training_mean

# ==============================================================================
# 3. Define Forecasting Input Sequence and Ground Truth Target
# ==============================================================================
# Use the last 48 hours from the scaled training features to form the forecasting input.
forecast_sequence = training_features[-sequence_length:]   # Shape: (48, 1)
forecast_sequence = np.expand_dims(forecast_sequence, axis=0)  # Shape: (1, 48, 1)

# The ground truth target is taken from the reserved forecast_data.
test_target = forecast_features.flatten()  # Shape: (48,)

# ==============================================================================
# 4. Custom Objects for Keras Model Loading (if needed)
# ==============================================================================
class GetItem(tf.keras.layers.Layer):
    def __init__(self, index=slice(None, None, None), **kwargs):
        super(GetItem, self).__init__(**kwargs)
        self.index = index

    def call(self, inputs):
        return inputs[self.index]

    def get_config(self):
        config = super(GetItem, self).get_config()
        config.update({"index": (self.index.start, self.index.stop, self.index.step)})
        return config

def r2(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - SS_res/(SS_tot + tf.keras.backend.epsilon())

def clipped_mape(y_true, y_pred):
    epsilon = 1e-3
    y_true_clipped = tf.maximum(y_true, epsilon)
    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true_clipped)) * 100

custom_objects = {
    "clipped_mape": clipped_mape,
    "r2": r2,
    "r2_metric": tf.keras.metrics.MeanMetricWrapper(r2, name="r2"),
    "GetItem": GetItem
}

# ==============================================================================
# 5. Model Prediction on the Forecasting Set
# ==============================================================================
# Assumption: The following models are already trained/loaded.
# Replace these dummy model instances with your actual model objects.
hybrid_pred = hybrid_model.predict(forecast_sequence)[0].flatten()       # Proposed iTBG-Net
cnn_pred = cnn_model.predict(forecast_sequence)[0].flatten()             # CNN (Incremental)
lstm_inc_pred = lstm_model.predict(forecast_sequence)[0].flatten()       # LSTM (Incremental)

# For the XLSTM incremental model (PyTorch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelxlstm.to(device)
test_tensor = torch.tensor(forecast_sequence, dtype=torch.float32).to(device)
modelxlstm.eval()
with torch.no_grad():
    xlstm_pred = modelxlstm(test_tensor).cpu().numpy().flatten()

# ==============================================================================
# 6. Compute Evaluation Metrics (MAE, MSE)
# ==============================================================================
def compute_metrics(forecast, actual):
    forecast = forecast.flatten()
    actual = actual.flatten()
    mae = np.mean(np.abs(forecast - actual))
    mse = np.mean((forecast - actual)**2)
    return mae, mse

hybrid_mae, hybrid_mse = compute_metrics(hybrid_pred, test_target)
cnn_mae, cnn_mse = compute_metrics(cnn_pred, test_target)
lstm_inc_mae, lstm_inc_mse = compute_metrics(lstm_inc_pred, test_target)
xlstm_mae, xlstm_mse = compute_metrics(xlstm_pred, test_target)

# Create a DataFrame to compare the MAE and MSE of the models.
df_metrics = pd.DataFrame({
    "Model": ["Proposed iTBG-Net", "CNN (Incremental)", "LSTM (Incremental)", "XLSTM (Incremental)"],
    "MAE": [hybrid_mae, cnn_mae, lstm_inc_mae, xlstm_mae],
    "MSE": [hybrid_mse, cnn_mse, lstm_inc_mse, xlstm_mse]
})
print(df_metrics)

# ==============================================================================
# 7. Visualization: Bar Plots for MAE and MSE on the Forecasting Set with Annotations
# ==============================================================================
x = np.arange(len(df_metrics["Model"]))
bar_width = 0.5
colors = ["#1f77b4", "#2ca02c", "#d62728", "#9467bd"]

# MAE Bar Plot
plt.figure(figsize=(6, 4), dpi=300)
rects = plt.bar(x, df_metrics["MAE"], color=colors, width=bar_width, edgecolor="black")
plt.title("MAE Comparison on Forecasting Set", fontsize=10, fontweight="bold")
plt.ylabel("MAE", fontsize=10, fontweight="bold")
plt.xticks(x, df_metrics["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.7)
# Annotate bars
for rect in rects:
    height = rect.get_height()
    plt.annotate(f'{height:.2f}',
                 xy=(rect.get_x() + rect.get_width() / 2, height),
                 xytext=(0, 3),  # Vertical offset
                 textcoords="offset points",
                 ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.savefig("mae_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

# MSE Bar Plot
plt.figure(figsize=(6, 4), dpi=300)
rects = plt.bar(x, df_metrics["MSE"], color=colors, width=bar_width, edgecolor="black")
plt.title("MSE Comparison on Forecasting Set", fontsize=10, fontweight="bold")
plt.ylabel("MSE", fontsize=10, fontweight="bold")
plt.xticks(x, df_metrics["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.7)
# Annotate bars
for rect in rects:
    height = rect.get_height()
    plt.annotate(f'{height:.2f}',
                 xy=(rect.get_x() + rect.get_width() / 2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.savefig("mse_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

# ==============================================================================
# 8. Additional Accuracy Bar Plot for Model Comparison with Annotations
# ==============================================================================
# Define a simple accuracy metric: percentage of predictions with error below a chosen tolerance.
def compute_accuracy(pred, actual, tol=0.1):
    # Returns the accuracy (%) where each prediction is considered accurate if its absolute error is less than tol.
    acc = np.mean(np.abs(pred - actual) < tol) * 100
    return acc

hybrid_acc = compute_accuracy(hybrid_pred, test_target)
cnn_acc = compute_accuracy(cnn_pred, test_target)
lstm_inc_acc = compute_accuracy(lstm_inc_pred, test_target)
xlstm_acc = compute_accuracy(xlstm_pred, test_target)

df_accuracy = pd.DataFrame({
    "Model": ["Proposed iTBG-Net", "CNN (Incremental)", "LSTM (Incremental)", "XLSTM (Incremental)"],
    "Accuracy (%)": [hybrid_acc, cnn_acc, lstm_inc_acc, xlstm_acc]
})
print(df_accuracy)

plt.figure(figsize=(6, 4), dpi=300)
rects = plt.bar(x, df_accuracy["Accuracy (%)"], color=colors, width=bar_width, edgecolor="black")
plt.title("Forecasting Accuracy Comparison", fontsize=10, fontweight="bold")
plt.ylabel("Accuracy (%)", fontsize=10, fontweight="bold")
plt.xticks(x, df_accuracy["Model"], rotation=15, fontsize=10, fontweight="bold")
plt.ylim(0, 100)
plt.grid(axis="y", linestyle="--", alpha=0.7)
# Annotate bars
for rect in rects:
    height = rect.get_height()
    plt.annotate(f'{height:.2f}',
                 xy=(rect.get_x() + rect.get_width() / 2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.savefig("accuracy_comparison.png", dpi=300, bbox_inches="tight")
plt.show()
"""

After **Running this code for three times and get the values** from the training time and the accuracy, making the statistical error plot

In [ ]:
"""import numpy as np
import matplotlib.pyplot as plt

# Data Taken After running complete notebook for three times and noted values
accuracy_data = {
    "iTBG-Net": [91.68, 90.92, 91.68],
    "CNN (IL)": [87.14, 87.14, 87.14],
    "LSTM (IL)": [87.77, 52.87, 87.77],
    "LSTM (Non-IL)": [50.27, 49.07, 50.27],
    "XLSTM (IL)": [89.62, 90.55, 89.62],
}

model_colors = {
    "iTBG-Net": "#1f77b4",
    "CNN (IL)": "#2ca02c",
    "LSTM (IL)": "#d62728",
    "LSTM (Non-IL)": "#ff7f0e",
    "XLSTM (IL)": "#9467bd",
}

# Mean & Std
mean_accuracies = {m: np.mean(v) for m, v in accuracy_data.items()}
std_devs = {m: np.std(v) for m, v in accuracy_data.items()}
models = list(mean_accuracies.keys())
means = list(mean_accuracies.values())
errors = [std_devs[m] for m in models]

# Plot
plt.figure(figsize=(6, 4), dpi=600)
bars = plt.bar(models, means, yerr=errors, capsize=5,
               color=[model_colors[m] for m in models],
               edgecolor='black', linewidth=1.2, width=0.55, alpha=0.85)

max_height = max([m + e for m, e in zip(means, errors)])
text_offset = max_height * 0.04  # More spacing above error bars

for i, bar in enumerate(bars):
    height = bar.get_height()
    err = errors[i]
    plt.text(bar.get_x() + bar.get_width() / 2,
             height + err + text_offset,
             f'{height:.2f} ± {err:.2f}', ha='center', va='bottom',
             fontsize=9, fontweight='bold', color='black')

plt.ylim(0, max_height * 1.25)
plt.ylabel("Forecasting Accuracy (%)", fontsize=10, fontweight="bold", labelpad=10)
#plt.title("48-Hour Ahead Forecasting Accuracy Comparison", fontsize=10, fontweight="bold", pad=10)
plt.xticks(rotation=15, fontsize=10, fontweight="bold")

# Only horizontal gridlines
plt.grid(axis="y", linestyle="--", linewidth=0.6, alpha=0.7)

plt.tight_layout()

# Save in high-res formats
plt.savefig("Forecasting_Accuracy_Std.png", dpi=600, bbox_inches="tight")
plt.savefig("Forecasting_Accuracy_Std.eps", dpi=600, bbox_inches="tight")
plt.savefig("Forecasting_Accuracy_Std.pdf", dpi=600, bbox_inches="tight")
plt.show()
"""

**Training Time after 3 time run and count the values**

In [ ]:
"""import numpy as np
import matplotlib.pyplot as plt

# Data Taken After running complete notebook for three times and noted values
training_time_data = {
    "iTBG-Net": [588.0, 636.9, 588.0],
    "CNN (IL)": [484.2, 539.7, 484.2],
    "LSTM (IL)": [468.5, 554.4, 468.5],
    "LSTM (Non-IL)": [1022.2, 1157.6, 1022.2],
    "XLSTM (IL)": [65.3, 69.4, 65.3],
}

model_colors = {
    "iTBG-Net": "#1f77b4",
    "CNN (IL)": "#2ca02c",
    "LSTM (IL)": "#d62728",
    "LSTM (Non-IL)": "#ff7f0e",
    "XLSTM (IL)": "#9467bd",
}

# Mean & Std
mean_times = {m: np.mean(v) for m, v in training_time_data.items()}
std_devs = {m: np.std(v) for m, v in training_time_data.items()}
models = list(mean_times.keys())
means = list(mean_times.values())
errors = [std_devs[m] for m in models]

# Plot
plt.figure(figsize=(6, 4), dpi=600)
bars = plt.bar(models, means, yerr=errors, capsize=5,
               color=[model_colors[m] for m in models],
               edgecolor='black', linewidth=1.2, width=0.55, alpha=0.85)  # Slightly increased width

max_height = max([m + e for m, e in zip(means, errors)])
text_offset = max_height * 0.04  # Offset for vertical spacing

for i, bar in enumerate(bars):
    height = bar.get_height()
    err = errors[i]
    plt.text(bar.get_x() + bar.get_width() / 2,
             height + err + text_offset,
             f'{height:,.1f} ± {err:.1f}',
             ha='center', va='bottom',
             fontsize=9, fontweight='bold', color='black')

# Axis settings
plt.ylim(0, max_height * 1.25)
plt.ylabel("Training Time (seconds)", fontsize=10, fontweight="bold", labelpad=10)
#plt.title("Training Time Comparison", fontsize=10, fontweight="bold", pad=10)
plt.xticks(rotation=15, fontsize=10, fontweight="bold")

# Only horizontal gridlines
plt.grid(axis="y", linestyle="--", linewidth=0.6, alpha=0.7)

plt.tight_layout()

# Save in high-res formats
plt.savefig("Training_Time_Std.png", dpi=600, bbox_inches="tight")
plt.savefig("Training_Time_Std.eps", dpi=600, bbox_inches="tight")
plt.savefig("Training_Time_Std.pdf", dpi=600, bbox_inches="tight")
plt.show()
"""

In [ ]:
import os
import zipfile
from IPython.display import FileLink

# Define the directory and output zip path
directory = "/kaggle/working/"
zip_filename = os.path.join(directory, "complete_plots.zip")

# Create the zip file
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(directory):
        # Only zip files in the root directory
        if root != directory:
            continue
        for file in files:
            if file == "complete_plots.zip":
                continue  # Skip the output zip itself
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, start=directory)
            zipf.write(file_path, arcname)

# Show download link
print("Zipping completed.")
FileLink(zip_filename)
comple